<a href="https://colab.research.google.com/github/BenKohn2004/Saber_Box/blob/master/Virtual_Saber_Box.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from os.path import exists, join, basename, splitext
from google.colab import drive
from google.colab import files
drive.mount('/content/drive', force_remount=True)
COLAB = True

# Determines if the Sync Folder is used for the Video Clip and Runs the Entire Sync Folder
run_most_recent_clip = False
run_entire_sync_folder = True
use_youtube_link = False

print("Note: using Google CoLab")

Mounted at /content/drive
Note: using Google CoLab


In [ ]:
# Leave youtube Link as '' to use saved clip
if use_youtube_link == True:
  youtube_link = 'https://youtu.be/9Ym1osNQGts?t=147'
else:
  youtube_link = ''

video_filename = '999.mp4'

In [ ]:
#@title Initial Parameters
# Establishes Initial Parameters
import time

# Provides additional comment feedback while running
verbose = False
# Crops each from to the tracking boxes
overlay_masking_boxes = False
# Adjusts the Reperesentative Dots based on camera motion implied by scoring box movement.
camera_motion_compensate = False
# Smooths the Bellguard positions
smooth_video_clip = False
# Assumes two lights if Bellguards are close to each other. Reduces dependency on Box detection.
assume_lights = True
# Ignores Lights from the Scorebox. Mitigates poor scorebox tracking.
ignore_box_lights = True
# Tests and Removes Duplicate Frames from the video
remove_duplicate_frames = False
# Downloads out and representative_out videos
download_videos = True
# Analyzes the Action
analyze_action = True
# Allows for Simple Usage
simplified = False
# Allows for the use of Exclusion Areas
use_Exclusion_Areas = True
# If the clip is longer than duration limit in seconds than a portion is downloaded instead of the whole clip
duration_download_limit = 2400
# Allows to run the entire clip or only till the lights turn on
run_entire_video = False
# Creates an Output video of the motion tracking
create_output_video = True
# Outputs the Bounding Box Data for the clip as a CSV file
output_bbox_clip = False
# Sets the output_bbox_clip to output the Foot Data
output_foot_data = False

min_torso_confidence = 0.70
bellguard_confidence = 0.40
foot_confidence = 0.60
# Provides for a higher confidence of bellguard detection
bellguard_confidence_high = 0.45
# Provides for a higher confidence of bellguard detection
bellguard_confidence_very_high = 0.50
# Provides for a higher confidence of bellguard detection
bellguard_confidence_extra_very_high = 0.55
# Allows for a different required confidence for initial detection than tracking
bellguard_tracking_det_offset = 0.15
wrist_conf_min = 3
wrist_conf_high = 4
wrist_conf_very_high = 6
wrist_conf_extra_very_high = 8
knee_conf_min = 3
# The Threshold for determining duplicate frames
duplicate_threshold_factor = 0.75
# The Threshold for determining camera motion
camera_motion_threshold_factor = 8
# Bellguard difference as a fraction of capture width to assume lights on
position_difference_ratio = .065
# Sets an upper limit for the size in kiloBytes of a file to be used with run_most_recent_clip == True
size_max = 4000

In [ ]:
#@title Imports
from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
import pandas as pd
from xml.etree import ElementTree
from PIL import Image
# from mrcnn.utils import Dataset
from matplotlib.patches import Rectangle
import random
import math
import glob
import cv2
import numpy as np
from numpy import random
from numpy import zeros
from numpy import asarray
import sys
import statistics
import PIL
import torchvision
import torch
torch.set_grad_enabled(False)
import matplotlib
import matplotlib.pylab as plt
import traceback
import os
from os import listdir
import shutil
# from shutil import copyfile
from scipy import signal
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.models import load_model
from numpy import expand_dims
from numpy import mean
from IPython.display import display, Javascript, Image, HTML
import matplotlib.pyplot as pyplot
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import io
import html

import imgaug  # https://github.com/aleju/imgaug (pip3 install imageaug)

if use_youtube_link == True:
  !pip install youtube_dl
  import youtube_dl

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

# if already_made == False:
# change makefile to have GPU, OPENCV and LIBSO enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

!make

fatal: destination path 'darknet' already exists and is not an empty directory.
[Errno 20] Not a directory: 'darknet'
/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: w

In [ ]:
# Copies over Weights, data file, configuration file and two name files.
%cd '/content/'

# Weights File
# Loads the file into Content
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1SFR5Hrt3LqhzPjfrG-kwPYO5Yy2fg7C8' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1SFR5Hrt3LqhzPjfrG-kwPYO5Yy2fg7C8" -O custom-yolov4-detector_best.weights && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mxfy7AeCbNs6-u3f606eh0NBCF4BRUzO' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1mxfy7AeCbNs6-u3f606eh0NBCF4BRUzO" -O custom-yolov4-detector_best.weights && rm -rf /tmp/cookies.txt
# Moves the file from Content to Destination
%cp custom-yolov4-detector_best.weights /content/darknet/custom-yolov4-detector_best.weights

# Object Data File
# Loads the file into Content
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mo394Soqlfyza9I-tk73D8LfzaBu2Qte' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1mo394Soqlfyza9I-tk73D8LfzaBu2Qte" -O obj_4.data && rm -rf /tmp/cookies.txt
# Moves the file from Content to Destination
%cp obj_4.data /content/darknet/cfg/obj_4.data

# Detection Config File
# Loads the file into Content
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-4fEmuJ542QdQC9w33BNTw8mAbDVWjqW' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-4fEmuJ542QdQC9w33BNTw8mAbDVWjqW" -O custom-yolov4-detector.cfg && rm -rf /tmp/cookies.txt
# Moves the file from Content to Destination
%cp custom-yolov4-detector.cfg /content/darknet/cfg/custom-yolov4-detector.cfg

# Coco Names File
# Loads the file into Content
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-3iHBsrWNuQaeSFoaaSIsb3uE4-mBRlG' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-3iHBsrWNuQaeSFoaaSIsb3uE4-mBRlG" -O coco.names && rm -rf /tmp/cookies.txt
# Moves the file from Content to Destination

%cp coco.names darknet/data/coco.names
%cp coco.names darknet/data/obj.names

/content
--2021-05-04 03:10:14--  https://docs.google.com/uc?export=download&confirm=IMkw&id=1mxfy7AeCbNs6-u3f606eh0NBCF4BRUzO
Resolving docs.google.com (docs.google.com)... 172.217.218.100, 172.217.218.102, 172.217.218.139, ...
Connecting to docs.google.com (docs.google.com)|172.217.218.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-0g-docs.googleusercontent.com/docs/securesc/7ph7v9s3j11o06c1q4nlaktdvp27emso/0dsn0fmmlilvtfgksf0i954u37noq9u5/1620097800000/15469335548275755091/10792393205273674053Z/1mxfy7AeCbNs6-u3f606eh0NBCF4BRUzO?e=download [following]
--2021-05-04 03:10:14--  https://doc-10-0g-docs.googleusercontent.com/docs/securesc/7ph7v9s3j11o06c1q4nlaktdvp27emso/0dsn0fmmlilvtfgksf0i954u37noq9u5/1620097800000/15469335548275755091/10792393205273674053Z/1mxfy7AeCbNs6-u3f606eh0NBCF4BRUzO?e=download
Resolving doc-10-0g-docs.googleusercontent.com (doc-10-0g-docs.googleusercontent.com)... 173.194.79.132, 2a00:1450:4013:c05::

In [ ]:
%cd darknet
# from darknet import *
!python3 darknet.py

# import darknet functions to perform object detections
from darknet import *

# Loads YOLOv4 Architecture Network
network, class_names, class_colors = load_network("cfg/custom-yolov4-detector.cfg", "cfg/obj_4.data", "custom-yolov4-detector_best.weights")
width = network_width(network)
height = network_height(network)

/content/darknet


In [ ]:
#@title Hour Minute Second String
def hms_string(sec_elapsed):
  # Nicely formatted time string
  h = int(sec_elapsed / (60 * 60))
  m = int((sec_elapsed % (60 * 60)) / 60)
  s = int(sec_elapsed % 60)
  ms = int(sec_elapsed * 1000 % 1000)
  return f"{h}:{m:>02}:{s:>02}:{ms:>03}"

In [ ]:
#@title Position_Linear_Approximation
def position_linear_approximation(position, previous_certainty):
  # Certainty is the number of times previous to current position that a point was not certain.
  last_known_position = ((previous_certainty+2)*(-1))

  # Finds the positional distance between two known boxes
  x_delta = int((position[-1][0] - position[last_known_position][0])/(last_known_position+1))
  y_delta = int((position[-1][1] - position[last_known_position][1])/(last_known_position+1))
  delta = [x_delta, y_delta]

  # Adjusts the previous positions, up to the previous certainty, based on a linear approximation
  for j in range(2):
    for i in range(previous_certainty+1):
      position[i - (previous_certainty+1)][j] = position[i - (previous_certainty+2)][j] - delta[j]

  return (position)

In [ ]:
#@title Scoring_Box_Lights
def scoring_box_lights(img, Scoring_Box_Position, scoring_box_size_average, default_color, frame, score_box_empty):

  # A high max distance is less sensitive and a lower max distance is more sensitive
  max_distance_total = 200
  max_distance_specific_color = 100

  # Defines the region of the top_left position of a 5x3 grid of the score_box, [xmin,ymin,xmax,ymax]
  # Extends the Light Search Position outside of the detected box
  xmin = Scoring_Box_Position[0] - int(scoring_box_size_average[0]/2) - int(scoring_box_size_average[0]/8)
  xmax = Scoring_Box_Position[0] - int(scoring_box_size_average[0]/2) + int(scoring_box_size_average[0]/4)
  ymin = Scoring_Box_Position[1] - int(scoring_box_size_average[1]/2)
  ymax = Scoring_Box_Position[1] - int(scoring_box_size_average[1]/2) + int(scoring_box_size_average[1]/3)
  left_light_position = [xmin, xmax, ymin, ymax]

  # Defines the region of the top_right position of a 5x3 grid of the score_box, [xmin,ymin,xmax,ymax]
  xmin = Scoring_Box_Position[0] + int(scoring_box_size_average[0]/2) - int(scoring_box_size_average[0]/4)
  xmax = Scoring_Box_Position[0] + int(scoring_box_size_average[0]/2) + int(scoring_box_size_average[0]/8)
  ymin = Scoring_Box_Position[1] - int(scoring_box_size_average[1]/2)
  ymax = Scoring_Box_Position[1] - int(scoring_box_size_average[1]/2) + int(scoring_box_size_average[1]/3)
  right_light_position = [xmin, xmax, ymin, ymax]

  if default_color != []:
    distance_temp, distance_specific_color_temp = [], []

    width = left_light_position[1]-left_light_position[0]
    height = left_light_position[3]-left_light_position[2]

    #i is the x value of the image for the Left Side/Red
    for i in range(width):
      #j is y value of the image
      for j in range(height):
        #color channel of the image [B,G,R]
        #image, img, is of format [y,x]
        pixel_position_y = left_light_position[2] + j
        pixel_position_x = left_light_position[0] + i
        b = (img[pixel_position_y, pixel_position_x, 0] - default_color[0])
        g = (img[pixel_position_y, pixel_position_x, 1] - default_color[1])
        r = (img[pixel_position_y, pixel_position_x, 2] - default_color[2])
        distance_temp.append(int((b**2 + g**2 + r**2)**(0.5)))
        distance_specific_color_temp.append(abs(r))

    #Sorts the distances and keeps the top quarter then finds the average
    distance_temp.sort()
    distance_temp = distance_temp[(int(len(distance_temp)/4)*-1):]
    distance = int(sum(distance_temp)/len(distance_temp))
    distance_specific_color_temp.sort()
    distance_specific_color_temp = distance_specific_color_temp[(int(len(distance_specific_color_temp)/4)*-1):]
    distance_specific_color = int(sum(distance_specific_color_temp)/len(distance_specific_color_temp))

    #0 is no color change from the default color)
    if distance > max_distance_total and distance_specific_color > max_distance_specific_color and score_box_empty == False:
      left_light_comparison = 1
    #1 is a color change from the default color
    else:
      left_light_comparison = 0

    #Resets b,g,r for the Right Side
    distance_temp, distance_specific_color_temp= [], []
    width = right_light_position[1]-right_light_position[0]
    height = right_light_position[3]-right_light_position[2]

    #i is the x value of the image
    for i in range(width):
      #j is y value of the image
      for j in range(height):
        #kcolor channel of the image [B,G,R]

        # pixel_position = right_light_position[2] + j,right_light_position[0] + i
        pixel_position_y = right_light_position[2] + j
        pixel_position_x = right_light_position[0] + i
        b = (img[pixel_position_y, pixel_position_x, 0] - default_color[0])
        g = (img[pixel_position_y, pixel_position_x, 1] - default_color[1])
        r = (img[pixel_position_y, pixel_position_x, 2] - default_color[2])
        distance_temp.append(int((b**2 + g**2 + r**2)**(0.5)))
        distance_specific_color_temp.append(abs(g))

    #Sorts the distances and keeps the top sixth then finds the average
    distance_temp.sort()
    distance_temp = distance_temp[(int(len(distance_temp)/6)*-1):]
    distance = int(sum(distance_temp)/len(distance_temp))
    distance_specific_color_temp.sort()
    distance_specific_color_temp = distance_specific_color_temp[(int(len(distance_specific_color_temp)/4)*-1):]
    distance_specific_color = int(sum(distance_specific_color_temp)/len(distance_specific_color_temp))

    #0 is no color change from the default color)
    if (distance > max_distance_total and distance_specific_color > max_distance_specific_color):
      right_light_comparison = 1
    #1 is a color change from the default color
    else:
      right_light_comparison = 0

  #Finds the Defualt Color
  else:
    b, g, r = 0, 0, 0
    # Cycles through the Left and Right Light Positions to determine a default color for the frame
    width = left_light_position[1]-left_light_position[0]
    height = left_light_position[3]-left_light_position[2]
    for i in range(width):
      for j in range(height):
        pixel_position_y = left_light_position[2] + j
        pixel_position_x = left_light_position[0] + i
        b = b + img[pixel_position_y, pixel_position_x, 0]
        g = g + img[pixel_position_y, pixel_position_x, 1]
        r = r + img[pixel_position_y, pixel_position_x, 2]
        default_color_left_temp = [int(b/(width*height)),int(g/(width*height)),int(r/(width*height))]
    width = right_light_position[1]-right_light_position[0]
    height = right_light_position[3]-right_light_position[2]
    for i in range(width):
      for j in range(height):
        # pixel_position = right_light_position[2] + j,right_light_position[0] + i
        pixel_position_y = left_light_position[2] + j
        pixel_position_x = left_light_position[0] + i
        b = b + img[pixel_position_y, pixel_position_x, 0]
        g = g + img[pixel_position_y, pixel_position_x, 1]
        r = r + img[pixel_position_y, pixel_position_x, 2]
        default_color_right_temp = [int(b/(width*height)),int(g/(width*height)),int(r/(width*height))]
    #Combines the Left and Right Default Colors for B,G,R
    for i in range(3):
      default_color.append((default_color_left_temp[i] + default_color_right_temp[i])/2)

    # Assumes that the lights are off during the engarde phase.
    left_light_comparison = 0
    right_light_comparison = 0

  return (left_light_comparison, right_light_comparison, default_color)

In [ ]:
#@title Motion_Difference_Tracking
def motion_difference_tracking(frame, side, Bounding_Box, width, height, kernel_scaling, erosion_iterations, dilation_iterations, orig_img_worpt_starting_list):

  # Ensures Bounding_Box is not negative
  for i in range(len(Bounding_Box)):
    if Bounding_Box[i] < 0:
      Bounding_Box[i] = 0

  if verbose == True:
    display(f'The original difference tracking bounding box at frame {frame - 1} is:')
    display(Bounding_Box)

  # Requires the Bounding Box to have a width and be on the image
  if Bounding_Box[1] - Bounding_Box[0] != 0 and Bounding_Box[0] < width and Bounding_Box[0] > 0:
    Position_y_Orig = int((Bounding_Box[3]+Bounding_Box[2])/2)

    # Reads the images
    image1 = orig_img_worpt_starting_list[frame - 1]
    image2 = orig_img_worpt_starting_list[frame - 2]

    # image1 = cv2.imread(file_name1)
    # image2 = cv2.imread(file_name2)

    # Convert to Grayscale
    image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    image_diff = cv2.absdiff(image1_gray,image2_gray)

    # Creates a Cropped Image
    crop_img = image_diff[Bounding_Box[2]:Bounding_Box[3], Bounding_Box[0]:Bounding_Box[1]]

    # Kernel is affected by Kernel Scaling which gets finer if it initially fails
    kernel_number = int(width/(100*kernel_scaling))
    
    # Ensures that the kernel is odd
    if kernel_number%2 == 0:
      kernel_number = kernel_number + 1
    kernel = np.ones((kernel_number,kernel_number),np.uint8)
    
    if crop_img.shape[0] != 0 and crop_img.shape[1] != 0:

      # Errodes
      erosion = cv2.erode(crop_img,kernel,iterations = erosion_iterations)

      # Dilates
      dilation = cv2.dilate(erosion,kernel,iterations = dilation_iterations)

      # Blurs Image
      blur = cv2.GaussianBlur(dilation,kernel.shape,0)

      # Threshold
      ret,thresh = cv2.threshold(blur,0,90,cv2.THRESH_BINARY)

      # Find contours
      cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      cnts = cnts[0] if len(cnts) == 2 else cnts[1]
      
      if cnts != []:
        c = max(cnts, key=cv2.contourArea)

        left = tuple(c[c[:, :, 0].argmin()][0])
        right = tuple(c[c[:, :, 0].argmax()][0])

        if verbose == True:
          display(f'Left/Right is {left[0]}/{right[0]}.')
          display(f'The int(left[0]) is {int(left[0])} and of type {type(int(left[0]))} while left[0] is {type(left[0])}.')
          display(f'Bounding_Box[0] is {Bounding_Box[0]} and is of {type(Bounding_Box[0])}.')

        if side == 'Left':
          # Obtain outer left coordinate of the contour
          # right = tuple(c[c[:, :, 0].argmin()][0])
          position = [int(right[0]) + Bounding_Box[0], Position_y_Orig]
        elif side == 'Right':
          # left = tuple(c[c[:, :, 0].argmax()][0])
          position = [int(left[0]) + Bounding_Box[0], Position_y_Orig]
        else:
          if verbose == True:
            display(f'Side is not given')

      
      else:
        if verbose == True:
          display(f'There is no data from difference imaging on the {side} side.')
        position = 'None'

    else:
      if verbose == True:
        display(f'The crop image is null on the {side} side.')
      position = 'None'
      cnts = []

    if verbose == True:
      display(f'The kernel number for frame {frame} is {kernel_number}, the number of errosions/dilations are {erosion_iterations}/{dilation_iterations}.')
    if cnts != []:
      if verbose == True:
        display(f'The resulting position is {position} and the boundary box is {Bounding_Box}. The Left/Right limits of the contour are {int(left[0]) + Bounding_Box[0]}/{int(right[0]) + Bounding_Box[0]}.')
  else:
    position = 'None'
    if verbose == True:
      display(f'The bounding box given had a width of zero.')

  return(position)

In [ ]:
#@title Saturation_Test
def saturation_test(box, frame, save_image_list):
  # Test is a True/False return
  # Takes an image and tests it for the expected saturation

  img = save_image_list[frame]
  # Converts from BGR to HSV
  img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
  
  # Tests Bellguard
  if box[2] == 1:
    blue_range = [50, 150]
    green_range = [50, 150]
    red_range = [50, 160]
    max_delta = 25
    # saturation_range = [0, 20]
    saturation_range = [0, 70]
    object_tested = 'Bellguard'
  # Tests Torso
  elif box[2] == 3:
    blue_range = [60, 150]
    green_range = [60, 150]
    red_range = [60, 160]
    max_delta = 30
    saturation_range = [0, 20]
    object_tested = 'Torso'
  else:
    if verbose == True:
      display(f'The object to test does not have a color/saturation profile.')

  width = (box[0][3]-box[0][1])
  height = (box[0][2]-box[0][0])

  s_temp = []

  #i is the x value of the image
  for i in range(width):
    #j is y value of the image
    for j in range(height):
      s = img[box[0][0] + j, box[0][1] + i, 1]
      s_temp.append(s)

    #Sorts the distances and keeps the top quarter then finds the average
    s_temp.sort()
    #Truncates to the least saturated/most gray values
    s_temp = s_temp[:(int(len(s_temp)/2)*-1)]
    s_temp = s_temp[:(int(len(s_temp)*3/4)*-1)]
    #Averages the saturation values
    s_average = int(sum(s_temp)/len(s_temp))

  if s_average < saturation_range[1]:
    test_result = True
  else:
    test_result = False

  if verbose == True:
    display(f'The test result for the {object_tested} saturation is {test_result} with a saturation of {s_average}.')

  return (test_result)

In [ ]:
def exclusion_area_simplification(a, max_dist):
  a_simplified = []
  for i in range(len(a)):
    for j in range(len(a) - (i+1)):
      # a[i] - a[(i+1)+j]
      if a[i] != 'skip' and a[(i+1)+j] != 'skip':
        dist = int(((a[i][0] - a[(i+1)+j][0])**2 + (a[i][1] - a[(i+1)+j][1])**2)**(0.5))
        # display(f'The distance between {a[i]} and {a[(i+1)+j]} is {dist}.')
        if dist < max_dist:
          a_simplified.append(a[i])
          a[i] = 'skip'
          a[(i+1)+j] = 'skip'

  for k in range(len(a)):
    if a[k] != 'skip':
      a_simplified.append(a[k])

  return(a_simplified)

In [ ]:
def exclusion_area_simplification_recursion(a, max_dist):
  a_simplified = exclusion_area_simplification(a, max_dist)
  a_simplified_temp = 0

  while a_simplified != a_simplified_temp:
    if verbose == True:
      display(a_simplified)
    a_simplified_temp = exclusion_area_simplification(a_simplified, max_dist)
    if a_simplified_temp != a_simplified:
      a_simplified = a_simplified_temp
      a_simplified_temp = 0

  if a_simplified_temp == a_simplified:
    if verbose == True:
      display(f'The two are equal.')
  else:
    if verbose == True:
      display(f'The two are NOT equal.')

  return(a_simplified)

In [ ]:
#@title Box_Size_Finder
def box_size_finder(bbox, capture_width, capture_height, object_to_size):

  Box_Size = [[],[]]
  sum_of_boxes = [[],[]]
  frame_multiplier = 1

  if object_to_size == 'score_box':
    x_min = int(capture_width/4)
    x_max = int(capture_width*3/4)
    bbox_category = 2
  elif object_to_size == 'left':
    x_min = 0
    x_max = int(capture_width/2)
    bbox_category = 3
  elif object_to_size == 'right':
    x_min = int(capture_width/2)
    x_max = int(capture_width)
    bbox_category = 3

  # i represents the frame, minimum of 50 frames or len(bbox)
  for i in range(min(50*frame_multiplier, len(bbox))):
    # j represents the rois(specific bounding box) within the frame sorted by confidence score
    for j in range(len(bbox[i])):
      if (bbox[i][j][1] > 0.90 and bbox[i][j][0][1] > x_min and bbox[i][j][0][1] < x_max and bbox[i][j][2] == bbox_category):
        #Appends x value:
        sum_of_boxes[0].append(bbox[i][j][0][1])
        #Appends y value:
        sum_of_boxes[1].append(bbox[i][j][0][0])  
        #Appends x width value:
        Box_Size[0].append(bbox[i][j][0][3] - bbox[i][j][0][1])
        #Appends y width value:
        Box_Size[1].append(bbox[i][j][0][2] - bbox[i][j][0][0])

  x_average = average_list(sum_of_boxes[0])
  y_average = average_list(sum_of_boxes[1])

  # scoring_box_size_average [Width, Height]
  box_size_average = []
  # Appends the average scoring box width
  box_size_average.append(int(average_list(Box_Size[0])))
  # Appends the average scoring box height
  box_size_average.append(int(average_list(Box_Size[1])))

  if verbose == True:
    display(f'The Average Box Size for {object_to_size} is {box_size_average}')

  return (box_size_average)

In [ ]:
#@title Tracking_Box_Default
def tracking_box_default(Left, Right, Score_Box, x_padding, y_padding, engarde_length):
  # Creates a default tracking box

  Tracking_Bounding_Boxes_Temp = [[],[],[]]
  Tracking_Bounding_Boxes = []

  for i in range(engarde_length):
    Tracking_Bounding_Boxes_Temp[0].append(Left[0] - x_padding)
    Tracking_Bounding_Boxes_Temp[0].append(Left[0] + x_padding)
    Tracking_Bounding_Boxes_Temp[0].append(Left[1] - y_padding)
    Tracking_Bounding_Boxes_Temp[0].append(Left[1] + y_padding)

    Tracking_Bounding_Boxes_Temp[1].append(Right[0] - x_padding)
    Tracking_Bounding_Boxes_Temp[1].append(Right[0] + x_padding)
    Tracking_Bounding_Boxes_Temp[1].append(Right[1] - y_padding)
    Tracking_Bounding_Boxes_Temp[1].append(Right[1] + y_padding)

    Tracking_Bounding_Boxes_Temp[2].append(Score_Box[0] - x_padding)
    Tracking_Bounding_Boxes_Temp[2].append(Score_Box[0] + x_padding)
    Tracking_Bounding_Boxes_Temp[2].append(Score_Box[1] - y_padding)
    Tracking_Bounding_Boxes_Temp[2].append(Score_Box[1]+ y_padding)

    Tracking_Bounding_Boxes.append(Tracking_Bounding_Boxes_Temp)

  return (Tracking_Bounding_Boxes)

In [ ]:
#@title Bell_Guard_Position_Finding
def Bell_Guard_Position_Finding(bbox, capture_width, capture_height, positions, frame_count, left_torso_size_average, right_torso_size_average, engarde_length, previous_certainty, camera_steady, camera_motion_threshold, Exclusion_Areas, orig_img_worpt_starting_list):
  # Format positions = [Left_Position, Right_Position, Score_Box_Position, Left_Torso_Position, Right_Torso_Position]

  x_min = []
  x_max = []
  y_min = []
  y_max = []

  Left_Position = positions[0]
  Right_Position = positions[1]
  Scoring_Box_Position = positions[2]
  Left_Torso_Position = positions[3]
  Right_Torso_Position = positions[4]
  Left_Foot_Position = positions[5]
  Right_Foot_Position = positions[6]

  # Any of the First engarde_length position can be used since the engarde position is an averaged constant
  # Certainty is used here as a counter for how many times a bounding box does not fall in the tracking box
  # And increases the size of the bounding box based on each miss

  certainty = [0,0,0,0,0,0,0]
  if verbose == True:
    display(f'Previous Certainty at frame {frame_count - 1} is {previous_certainty}.')

  #Establishes Previous Positions to determine speed and expected positions
  previous_position_Left = Left_Position[-1]
  twice_previous_position_Left = Left_Position[-2]
  previous_position_Right = Right_Position[-1]
  twice_previous_position_Right = Right_Position[-2]
  previous_position_Scoring_Box = Scoring_Box_Position[-1]
  twice_previous_position_Scoring_Box = Scoring_Box_Position[-2]
  previous_position_Left_Torso = Left_Torso_Position[-1]
  twice_previous_position_Left_Torso = Left_Torso_Position[-2]
  previous_position_Right_Torso = Right_Torso_Position[-1]
  twice_previous_position_Right_Torso = Right_Torso_Position[-2]
  previous_position_Left_Foot = Left_Foot_Position[-1]
  twice_previous_position_Left_Foot = Left_Foot_Position[-2]
  previous_position_Right_Foot = Right_Foot_Position[-1]
  twice_previous_position_Right_Foot = Right_Foot_Position[-2]

  #Boxes are the bounding boxes for the current frame, passes less data to tracking function
  boxes = bbox

  # Tracking_Bounding_Boxes_Temp = [[],[],[],[],[]]
  Tracking_Bounding_Boxes_Temp = [[],[],[],[],[],[],[]]

  # Torso Positions are calculated prior to the BellGuard because they are an input to the bellguard position

  # Bellguard Position Tracking focuses on Tracking as opposed to detection
  
  # Left_Torso Position
  [current_position, certainty[3], Tracking_Bounding_Boxes_Left_Torso] = \
    Bell_Guard_Position_Tracking(boxes, previous_position_Left_Torso, \
    twice_previous_position_Left_Torso, previous_certainty[3], 'Left_Torso', \
    frame_count, 'None', left_torso_size_average, capture_width, capture_height, \
    engarde_length, camera_steady, camera_motion_threshold, \
    'None', orig_img_worpt_starting_list)
  Tracking_Bounding_Boxes_Temp[3] = Tracking_Bounding_Boxes_Left_Torso
  Left_Torso_Position = current_position

  # Right_Torso Position
  [current_position, certainty[4], Tracking_Bounding_Boxes_Right_Torso] = \
    Bell_Guard_Position_Tracking(boxes, previous_position_Right_Torso, \
    twice_previous_position_Right_Torso, previous_certainty[4], "Right_Torso", \
    frame_count, 'None', right_torso_size_average, capture_width, capture_height, \
    engarde_length, camera_steady, camera_motion_threshold, \
    'None', orig_img_worpt_starting_list)
  Tracking_Bounding_Boxes_Temp[4] = Tracking_Bounding_Boxes_Right_Torso
  Right_Torso_Position = current_position

  # Left Position
  [current_position, certainty[0], Tracking_Bounding_Boxes_Left] = \
    Bell_Guard_Position_Tracking(boxes, previous_position_Left, \
    twice_previous_position_Left, previous_certainty[0], 'Left_BellGuard', \
    frame_count, Left_Torso_Position, left_torso_size_average, capture_width, \
    capture_height, engarde_length, camera_steady, camera_motion_threshold, \
    Exclusion_Areas, orig_img_worpt_starting_list)
  Tracking_Bounding_Boxes_Temp[0] = Tracking_Bounding_Boxes_Left
  Left_Position = current_position

  #  Right Position
  [current_position, certainty[1], Tracking_Bounding_Boxes_Right] = \
    Bell_Guard_Position_Tracking(boxes, previous_position_Right, \
    twice_previous_position_Right, previous_certainty[1], 'Right_BellGuard', \
    frame_count, Right_Torso_Position, right_torso_size_average, capture_width, \
    capture_height, engarde_length, camera_steady, camera_motion_threshold, \
    Exclusion_Areas, orig_img_worpt_starting_list)
  Tracking_Bounding_Boxes_Temp[1] = Tracking_Bounding_Boxes_Right
  Right_Position = current_position

  # Scoring_Box Position
  [current_position, certainty[2], Tracking_Bounding_Boxes_Scoring_Box] = \
    Bell_Guard_Position_Tracking(boxes, previous_position_Scoring_Box, \
    twice_previous_position_Scoring_Box, previous_certainty[2], 'Scoring_Box', \
    frame_count, 'None', left_torso_size_average, capture_width, capture_height, \
    engarde_length, camera_steady, camera_motion_threshold, \
    'None', orig_img_worpt_starting_list)
  Tracking_Bounding_Boxes_Temp[2] = Tracking_Bounding_Boxes_Scoring_Box
  Scoring_Box_Position = current_position

  # Left Foot Position
  [current_position, certainty[5], Tracking_Bounding_Boxes_Left_Foot] = \
    Bell_Guard_Position_Tracking(boxes, previous_position_Left_Foot, \
    twice_previous_position_Left_Foot, previous_certainty[5], 'Left_Foot', \
    frame_count, Left_Torso_Position, left_torso_size_average, capture_width, \
    capture_height, engarde_length, camera_steady, camera_motion_threshold, \
    'None', orig_img_worpt_starting_list)
  Tracking_Bounding_Boxes_Temp[5] = Tracking_Bounding_Boxes_Left_Foot
  Left_Foot_Position = current_position

  # Left Foot Position
  [current_position, certainty[5], Tracking_Bounding_Boxes_Left_Foot] = \
    Bell_Guard_Position_Tracking(boxes, previous_position_Left_Foot, \
    twice_previous_position_Left_Foot, previous_certainty[5], 'Left_Foot', \
    frame_count, Left_Torso_Position, left_torso_size_average, capture_width, \
    capture_height, engarde_length, camera_steady, camera_motion_threshold, \
    'None', orig_img_worpt_starting_list)
  Tracking_Bounding_Boxes_Temp[5] = Tracking_Bounding_Boxes_Left_Foot
  Left_Foot_Position = current_position

  # Right Foot Position
  [current_position, certainty[6], Tracking_Bounding_Boxes_Right_Foot] = \
    Bell_Guard_Position_Tracking(boxes, previous_position_Right_Foot, \
    twice_previous_position_Right_Foot, previous_certainty[6], 'Right_Foot', \
    frame_count, Right_Torso_Position, right_torso_size_average, capture_width, \
    capture_height, engarde_length, camera_steady, camera_motion_threshold, \
    'None', orig_img_worpt_starting_list)
  Tracking_Bounding_Boxes_Temp[6] = Tracking_Bounding_Boxes_Right_Foot
  Right_Foot_Position = current_position

  Tracking_Bounding_Boxes = Tracking_Bounding_Boxes_Temp

  if verbose == True:
    display(f'The Length of the Left and Right Positions after the Position Finding are: {len(Left_Position)} and {len(Right_Position)}.')
    display(f'At frame {frame_count} the certainty and previous certainty before linear approx analysis is:')
    display(f'{certainty} and {previous_certainty}')

  return (Left_Position, Right_Position, Scoring_Box_Position, Tracking_Bounding_Boxes, Left_Torso_Position, Right_Torso_Position, Left_Foot_Position, Right_Foot_Position, certainty)

In [ ]:
#@title Bell_Guard_Position_Tracking
def Bell_Guard_Position_Tracking(boxes, previous_position, twice_previous_position, certainty, tracked_item, frame, Torso_Position, Torso_Size, capture_width, capture_height, engarde_length, camera_steady, camera_motion_threshold, Exclusion_Areas, orig_img_worpt_starting_list):
  # Tracks the position of items
  # tracked_item is needed since boxes only have the class of the item tracked, not the Left or Right
  # tracked_item Format: [0,1,2,3] = [Background, Bell_Guard, Score_Box, Torso]

  #Assumed inherent uncertainty
  # certainty_default = int(capture_width/8)
  certainty_default = int(capture_width/12)
  certainty_multiplier = int(capture_width/80)

  #Reduces the max value of y as compared to x
  y_limiter = 24

  # Max allowed speed of a bellguard in a single frame
  # Accounts for a position jump following the engarde positioning
  if frame < engarde_length + 3:
    max_speed = int(capture_width/64)
  else:
    max_speed = int(capture_width/24)

  # Converts previous position into a speed
  x_pos = int(previous_position[0])
  if verbose == True:
    display(f'previous_position is {previous_position} and twice_previous_position is {twice_previous_position}.')
  # x_speed = int(min(previous_position[0] - twice_previous_position[0], max_speed))

  x_speed = previous_position[0] - twice_previous_position[0]
  if x_speed > 0:
    x_speed = min(x_speed, max_speed)
  else:
    x_speed = max(x_speed, max_speed*-1)

  y_pos = int(previous_position[1])
  y_speed = int(min(previous_position[1] - twice_previous_position[1], int(max_speed/y_limiter)))
  y_speed = int(max(y_speed, int(max_speed*(-1)/y_limiter)))

  if (frame - 1)  == engarde_length and verbose == True:
    display(f'THe x_speed is {x_speed} and the y_speed is {y_speed} at the engarde length, frame {frame - 1}.')

  # Flips the tracking box to be between the two fencers
  if tracked_item == 'Left_BellGuard' or tracked_item == 'Left_Torso' or tracked_item == 'Left_Foot':
    horiz_flip = False
    if verbose == True:
      display(f'The horizontal flip is {horiz_flip} for the {tracked_item} at frame {frame - 1}.')
  elif tracked_item == 'Right_BellGuard' or tracked_item == 'Right_Torso' or tracked_item == 'Right_Foot':
    horiz_flip = True
    if verbose == True:
      display(f'The horizontal flip is {horiz_flip} for the {tracked_item} at frame {frame - 1}.')
  else:
    horiz_flip = False

  # Defines the tracking box
  expected_position = [(x_pos + x_speed),(y_pos + y_speed)]

  # Allows for more lenient box following engarde positioning
  if frame < (engarde_length + 3):
    padding = int(certainty*certainty_multiplier + certainty_default*1.5)
  else:
    padding = int(certainty*certainty_multiplier + certainty_default)
  # boundary_box_for_tracking = [int(padding*7/8), padding, padding, padding]
  boundary_box_for_tracking = [int(padding*16/8), padding, padding, padding]
  if verbose == True:
    display(f'The Boundary Box for the {tracked_item} is {boundary_box_for_tracking} using a certainty of {certainty} with an expected position of {expected_position} at frame {frame - 1}')
  tracking_box = create_boundary_box(expected_position, boundary_box_for_tracking, horiz_flip)
  positions = []

  boxes_temp = []
  #Filters out potential boxes based on Tracked Item, Confidence and Saturation of the Box
  if (tracked_item == 'Left_BellGuard' or tracked_item == 'Right_BellGuard'):
    bell_certainty = certainty

    if verbose == True:
      display(f'The boxes being tested for exclusion are:')
      display(boxes)

    within_exclusion_box = False
    # Only Tests if Bell Guard confidence is not extra very high  
    for j in range(len(boxes)):
      if boxes[j][1] < bellguard_confidence_extra_very_high and use_Exclusion_Areas == True:
        # Tests if the box is in an exlusion area
        padding = capture_width/48
        exclusion_padding = [padding,padding,padding,padding]
        # Cycles through the Exclusion Areas
        for k in range(len(Exclusion_Areas)):
          if verbose == True:
            display(f'Exclusion_Areas[k] is {Exclusion_Areas[k]}.')
          Exculsion_Box = create_boundary_box(Exclusion_Areas[k],exclusion_padding, False)
          if verbose == True:
            display(f'boxes[j][0] is {boxes[j][0]}, boxes[j][1] is {boxes[j][1]}.')
          test_point = [int((boxes[j][0][3]+boxes[j][0][1])/2), int((boxes[j][0][2]+boxes[j][0][0])/2)]
          if verbose == True:
            display(f'The test_point is {test_point}.')
          within_tested_box = boundary_box_test(test_point, Exculsion_Box)
          if within_tested_box == True:
            within_exclusion_box = True
            if verbose == True:
              display(f'Within the exclusion box is {within_exclusion_box}.')
          else:
            if verbose == True:
              display(f'Within the exclusion box is {within_exclusion_box}.')

      # Allows Extra Very High Confidence regardless of exclusion areas
      if boxes[j][1] >= bellguard_confidence_extra_very_high:
         within_exclusion_box = False

      # for k in range(len(boxes)):
        #The minimum required certainty for a bellguard box
      # Appends the Appropriate boxes to the boxes list to become potential positions 
      if (boxes[j][2] == 1) and (boxes[j][1] > (bellguard_confidence - bellguard_tracking_det_offset)) and within_exclusion_box == False:
        boxes_temp.append(boxes[j])

  elif (tracked_item == 'Left_Torso' or tracked_item == 'Right_Torso'):
    for j in range(len(boxes)):
      # Tests the torso for a drastic change in height from engarde positioning
      torso_height = boxes[j][0][2] - boxes[j][0][0]
      if verbose == True:
        display(f'The Torso Height at frame {frame - 1}, region of interest {j} is {torso_height} with initial height of {Torso_Size[1]}.')

      if ((boxes[j][2] == 3) and (boxes[j][1] > min_torso_confidence) and torso_height > Torso_Size[1]*(2/3)):

        boundary_box = [int(Torso_Size[0]*4),int(Torso_Size[0]*4),int(Torso_Size[1]/4),int(Torso_Size[1]/4)]

        # boundary_box = [int(Torso_Size[0]),int(Torso_Size[0]*2),int(Torso_Size[1]/4),int(Torso_Size[1]/4)]                                 
        torso_box = create_boundary_box(expected_position, boundary_box, horiz_flip)                             
        # torso_box = create_boundary_box(Torso_Position, boundary_box_for_torso, horiz_flip)

        # Allows for more lenient box following engarde positioning
        if frame < (engarde_length + 3):
          [x_min, x_max, y_min, y_max] = tracking_box
        else:
          [x_min, x_max, y_min, y_max] = boundary_box_overlap(tracking_box, torso_box)
        x_center = int((boxes[j][0][1] + boxes[j][0][3])/2)
        y_center = int((boxes[j][0][0] + boxes[j][0][2])/2)
        torso_boundary_test = boundary_box_test([x_center,y_center], [x_min, x_max, y_min, y_max])
        if torso_boundary_test == True:
          boxes_temp.append(boxes[j])

  elif (tracked_item == 'Scoring_Box'):
    for j in range(len(boxes)):
      if (boxes[j][2] == 2):
        boxes_temp.append(boxes[j])

  elif (tracked_item == 'Left_Foot' or tracked_item == 'Right_Foot'):
    for j in range(len(boxes)):
      if ((boxes[j][2] == 4) and (boxes[j][1] > foot_confidence)):

        # Uses the Torso as a horizontal and vertical minimum
        boundary_box_for_torso = [0, int(Torso_Size[0]*2), -1*int(Torso_Size[1]), int(Torso_Size[1]*2.0)]
        torso_box = create_boundary_box(Torso_Position, boundary_box_for_torso, horiz_flip)
        [x_min, x_max, y_min, y_max] = boundary_box_overlap(tracking_box, torso_box)
        x_center = int((boxes[j][0][1] + boxes[j][0][3])/2)
        y_center = int((boxes[j][0][0] + boxes[j][0][2])/2)
        foot_boundary_test = boundary_box_test([x_center,y_center], [x_min, x_max, y_min, y_max])
        if foot_boundary_test == True:
          boxes_temp.append(boxes[j])

  # Assigns boxes_temp to boxes
  boxes = boxes_temp

  # Creates points at the centers of the bounding boxes that are in this frame
  x_center = []
  y_center = []
  for i in range(len(boxes)):
    x_center.append(int((boxes[i][0][1] + boxes[i][0][3])/2))
    y_center.append(int((boxes[i][0][0] + boxes[i][0][2])/2))

  if tracked_item == 'Left_BellGuard' or tracked_item == 'Right_BellGuard':
    # Sets the values for the torso boundary box, limits Bellguard distance from Torso center
    boundary_box_for_torso = [int(Torso_Size[0]*0.20), int(Torso_Size[0]*3.25), int(Torso_Size[1]*.75), int(Torso_Size[1]*1.0)]
    # Uses the boundary box to create a box based on Left/Right and expected/previous position
    torso_box = create_boundary_box(Torso_Position, boundary_box_for_torso, horiz_flip)
    # Finds the overlap of multiple boxes to satisy multiple restrictions
    [x_min, x_max, y_min, y_max] = boundary_box_overlap(tracking_box, torso_box)
    if verbose == True:
      display(f'The Torso_Size[0] is {Torso_Size[0]}, the Horizontal Flip is {horiz_flip} and Torso_Position is {Torso_Position}.')
  elif tracked_item == 'Left_Foot' or tracked_item == 'Right_Foot':
    # Uses the Torso as a horizontal and vertical minimum
    boundary_box_for_torso = [0, int(Torso_Size[0]*2), -1*int(Torso_Size[1]), int(Torso_Size[1]*2.0)]
    torso_box = create_boundary_box(Torso_Position, boundary_box_for_torso, horiz_flip)
    [x_min, x_max, y_min, y_max] = boundary_box_overlap(tracking_box, torso_box)
    if verbose == True:
      display(f'The Boundary Box for the {tracked_item} with the horiz_flip as {horiz_flip} at frame {frame -1} is:')
      display([x_min, x_max, y_min, y_max])
      display(f'The Torso Position is {Torso_Position}.')
      display(f'The Torso Box is {torso_box}.')
  else:
    [x_min, x_max, y_min, y_max] = tracking_box

  if verbose == True:
    display(f'The tracking box for the {tracked_item} at frame {frame - 1} is: {tracking_box}.')

  if (tracked_item == 'Left_BellGuard' or tracked_item == 'Right_BellGuard'):
    if verbose == True:
      display(f'The torso box for the {tracked_item} at frame {frame - 1} is: {torso_box}.')
      display(f'The overlapping tracking box for the {tracked_item} at frame {frame - 1} is: {[x_min, x_max, y_min, y_max]}.')

  # Creates a list of positions within the bounding boxes
  for i in range(len(boxes)):
    center = [x_center[i], y_center[i]]
    tracking_result = boundary_box_test(center,tracking_box)
    # If the center point is within both boxes for Bellguards or tracking box for other items, then it is appended to positions
    if tracked_item == 'Left_BellGuard' or tracked_item == 'Right_BellGuard':
      torso_result = boundary_box_test(center,torso_box)
      # if (frame - 1) == 49:
      #   display(f'For ({x_center[i]},{y_center[i]}), {boxes[i][1]}%, the tracking_result is {tracking_result} and the torso_result is {torso_result}.')
      # Allows for an incorrect engarde position for the bellguard
      if (frame - 1) > engarde_length + 3:
        if tracking_result == True and torso_result == True:
          positions.append([x_center[i],y_center[i], boxes[i][1]])
      else:
        # Only the torso results is required for the engarde positioning
        if torso_result == True:
          positions.append([x_center[i],y_center[i], boxes[i][1]])
    else:
      if tracking_result == True:
        positions.append([x_center[i],y_center[i], boxes[i][1]])

  # Maximum distance only applies if there are multiple bounding boxes within the tracking box
  maximum_distance_from_expected = int(capture_width/24)
  # Expected Position [x,y], Limits expected position in front of the fencer
  if tracked_item == 'Left_BellGuard' or tracked_item == 'Right_BellGuard':
    if verbose == True:
      display(f'The expected position is {expected_position} and Torso Position and size is {Torso_Position[0]} and {Torso_Size[0]}.')
    if (expected_position[0] > Torso_Position[0] + Torso_Size[0]*2.50) and tracked_item == 'Left_BellGuard':
      if verbose == True:
        display(f'At frame {frame - 1} the expected position of the {tracked_item} was too far in front of the Torso, adjusting expected.')
      expected_position = [int(Torso_Position[0] + Torso_Size[0]*2.50), y_pos]
    if (expected_position[0] < Torso_Position[0]) and tracked_item == 'Left_BellGuard':
      if verbose == True:
        display(f'At frame {frame - 1} the expected position of the {tracked_item} was behind the Torso, adjusting expected.')
      expected_position = [int(Torso_Position[0]), y_pos]
    if expected_position[0] < Torso_Position[0] - Torso_Size[0]*2.50 and tracked_item == 'Right_BellGuard':
      if verbose == True:
        display(f'At frame {frame - 1} the expected position of the {tracked_item} was too far from the Torso, adjusting expected.')
        display(f'Torso_Position[0] is {Torso_Position[0]}, Torso_Size[0] is {Torso_Size[0]}, y_pos is {y_pos}.')
      expected_position = [int(Torso_Position[0] - Torso_Size[0]*2.50), y_pos]
    if (expected_position[0] > Torso_Position[0]) and tracked_item == 'Right_BellGuard':
      if verbose == True:
        display(f'At frame {frame - 1} the expected position of the {tracked_item} was behind the Torso, adjusting expected.')
      expected_position = [int(Torso_Position[0]), y_pos]

  #Assumed maximum distance from wrist to bellguard
  wrist_to_bellguard_max = int(Torso_Size[0]/8)

  #Sets Initial Conditions for Type of Tracking
  using_human_pose = False
  using_difference_images = False
  using_difference_images_normal_kernel = False
  using_expected = False
  using_position = False

  # fencer_data

  if verbose == True:
    display(f'The camera steady value for frame {frame - 1} is {camera_steady[frame - 1]}.')
    if camera_steady[frame - 1] >= camera_motion_threshold:
      if verbose == True:
        display(f'The camera is in motion and motion detection is less reliable.')

  # Determines the Bellguard Position based on number of detections, confidence, box location and motion
  if (len(positions)) == 0:
    if verbose == True:
      display(f'There where no positions found for the {tracked_item} at frame {frame - 1}.')
    if tracked_item == 'Left_BellGuard' or tracked_item == 'Right_BellGuard':
      motion_difference_boundary = [int(Torso_Size[0]*5/8), int(Torso_Size[0] + 1), int(Torso_Size[0]/3), int(Torso_Size[0]/3)]
      if tracked_item == 'Left_BellGuard' and camera_steady[frame - 1] < camera_motion_threshold:
        boundary_box = create_boundary_box(expected_position, motion_difference_boundary, False)
        position = motion_difference_tracking(frame, 'Left', boundary_box, capture_width, capture_height, (capture_width/2560), 3, 4, orig_img_worpt_starting_list)
        box_test = boundary_box_test(position, boundary_box)
        if position != 'None' and box_test == True:
          using_difference_images_normal_kernel = True
        if position == 'None' or box_test == False:
          if verbose == True:
            display(f'Attempting to use a smaller kernel for motion difference tracking.')
          position = motion_difference_tracking(frame, 'Left', boundary_box, capture_width, capture_height, (capture_width/640), 3, 4, orig_img_worpt_starting_list)
          box_test = boundary_box_test(position, boundary_box)
          # if position != 'None' and box_test == True:
          #   using_difference_images_normal_kernel = True
          if position == 'None' or box_test == False:
            if verbose == True:
              display(f'Attempting to use a smallest kernel for motion difference tracking.')
            position = motion_difference_tracking(frame, 'Left', boundary_box, capture_width, capture_height, (capture_width/320), 3, 4, orig_img_worpt_starting_list)
            if position == 'None':
              if verbose == True:
                display(f'Using the far Left Portion of the tracking Box')
              position = [expected_position[0] - motion_difference_boundary[0], expected_position[1]]
        # Adjusts the position if motion detection is too far out from the Torso
        if position[0] > Torso_Position[0] + Torso_Size[0]*2.50:
          if verbose == True:
            display(f'The motion detected position was too far from the torso and was adjusted')
          position[0] = int(Torso_Position[0] + Torso_Size[0]*2.50)
          # position = [Torso_Position[0] + Torso_Size[0]*2.25, position[1]]
        if verbose == True:
          display(f'The position for motion difference frame {frame - 1} is ({position})')
          display(f'The boundary box test limits are {motion_difference_boundary} for frame {frame - 1}.')
        # boundary_box = create_boundary_box(expected_position, motion_difference_boundary, False)
        box_test = boundary_box_test(position, boundary_box)
        #Uses the Expected position if the motion difference is out of bounds
        if box_test == False:
          if verbose == True:
            display(f'Motion difference failed, using the Expected Position for the {tracked_item} for frame {frame - 1}.')
          position = expected_position
          using_expected = True
        else:
          if verbose == True:
            display(f'The motion difference position was used for the {tracked_item} at frame {frame - 1}.')
          using_difference_images = True
      elif tracked_item == 'Right_BellGuard' and camera_steady[frame - 1] < camera_motion_threshold:
        boundary_box = create_boundary_box(expected_position, motion_difference_boundary, True)
        position = motion_difference_tracking(frame, 'Right', boundary_box, capture_width, capture_height, (capture_width/2560), 3, 4, orig_img_worpt_starting_list)
        box_test = boundary_box_test(position, boundary_box)
        if position != 'None' and box_test == True:
          using_difference_images_normal_kernel = True
        if position == 'None' or box_test == False:
          if verbose == True:
            display(f'Attempting to use a smaller kernel for motion difference tracking.')
          position = motion_difference_tracking(frame, 'Right', boundary_box, capture_width, capture_height, (capture_width/640), 3, 4, orig_img_worpt_starting_list)
          box_test = boundary_box_test(position, boundary_box)
          # if position != 'None' and box_test == True:
          #   using_difference_images_normal_kernel = True
          if position == 'None' or box_test == False:
            if verbose == True:
              display(f'Attempting to use the smallest kernel for motion difference tracking.')
            position = motion_difference_tracking(frame, 'Right', boundary_box, capture_width, capture_height, (capture_width/320), 3, 4, orig_img_worpt_starting_list)
            # Uses the Right Portion of the Motion Tracking Box if no motion tracking is found
            if position == 'None' or box_test == False:
              if verbose == True:
                display(f'Using the far Right Portion of the tracking Box, ({expected_position[0] + motion_difference_boundary[0]},{expected_position[1]})')
              position = [expected_position[0] + motion_difference_boundary[0], expected_position[1]]
        if position[0] < Torso_Position[0] - Torso_Size[0]*2.50:
          if verbose == True:
            display(f'The motion detected position ({position[0]},{position[1]}) was too far from the torso ({Torso_Position[0]},{Torso_Position[1]}), with a max of {Torso_Position[0] - Torso_Size[0]*2.50} and was adjusted')
          position[0] = int(Torso_Position[0] - Torso_Size[0]*2.50)
          # position = [Torso_Position[0] - Torso_Size[0]*2.25, position[1]]
        if verbose == True:
          display(f'The position for motion difference frame {frame - 1} is ({position})')
          display(f'The boundary box test limits are {motion_difference_boundary} for frame {frame - 1}.')
        boundary_box = create_boundary_box(expected_position, motion_difference_boundary, True)
        box_test = boundary_box_test(position, boundary_box)
        # box_test = False
        if box_test == False:
          if verbose == True:
            display(f'Motion difference failed, using the Expected Position for the {tracked_item} for frame {frame - 1}.')
          position = expected_position
          using_expected = True
        else:
          if verbose == True:
            display(f'The motion difference position was used for the {tracked_item} at frame {frame - 1}.')
          using_difference_images = True
      else:
        if verbose == True:
          display(f'Too much camera motion, using expected position')
        position = expected_position
        using_expected = True    
    else:
      position = expected_position    

    # Criteria for Setting Certainty to zero preventing a linear appoximation adjustment of this point
    # Allows for motion difference to be certain if it uses the largest kernel
    if (using_difference_images_normal_kernel == True and position[1] < Torso_Position[1] + Torso_Size[1] and \
        abs(expected_position[0] - position[0]) < Torso_Size[0]/2 and camera_steady[frame - 1] < camera_motion_threshold):
      if using_difference_images == True:
        if verbose == True:
          display(f'Using difference images for frame {frame - 1} with no detected positions')
    # if (using_human_pose == True and fencer_data[0][2] > wrist_conf_high):
      certainty = 0
    else:
      certainty = certainty + 1

  # For a single detected Bellguard Position
  elif (len(positions)) == 1:
    if tracked_item == 'Left_BellGuard' or tracked_item == 'Right_BellGuard':
      if verbose == True:
        display(f'There is one possible position, {positions[0]} for {tracked_item} in the tracking box for frame {frame - 1}.')
      # Allows for a large bounding box if bell guard confidence is very high
      if positions[0][2] > bellguard_confidence_extra_very_high:
        if verbose == True:
          display(f'Using Bell Guard Extra Very High confidence.')
        single_position_box = [int(Torso_Size[0]*9/8*(1+bell_certainty/4)), int(Torso_Size[0]*9/8*(1+bell_certainty/4)), int(Torso_Size[0]*12/8), int(Torso_Size[0]*12/8)]
      elif positions[0][2] > bellguard_confidence_very_high and positions[0][2] <= bellguard_confidence_extra_very_high:
        if verbose == True:
          display(f'Using Bell Guard Very High confidence.')
        single_position_box = [int(Torso_Size[0]*6/8*(1+bell_certainty/4)), int(Torso_Size[0]*6/8*(1+bell_certainty/4)), int(Torso_Size[0]*12/8), int(Torso_Size[0]*12/8)]
      else:
        single_position_box = [int(Torso_Size[0]*4/8*(1+bell_certainty/4)), int(Torso_Size[0]*5/8*(1+bell_certainty/4)), int(Torso_Size[0]*8/8), int(Torso_Size[0]*8/8)]
      if tracked_item == 'Left_BellGuard':
        boundary_box = create_boundary_box(expected_position, single_position_box, False)
      else:
        boundary_box = create_boundary_box(expected_position, single_position_box, True)
      box_test = boundary_box_test(positions[0], boundary_box)
      if verbose == True:
        display(f'The expected position for frame {frame - 1} is {expected_position}.')
        display(f'The single_position_box is {single_position_box} and the boundary box is {boundary_box}.')
      if box_test == True and positions[0][2] > bellguard_confidence_high:
        if verbose == True:
          display(f'The detected position was used for the {tracked_item} at frame {frame - 1}.')
        position = positions[0]
        using_position = True
      else:
        #Human Pose
        if verbose == True:
          display(f'Attempting to use Human Pose for the {tracked_item} at frame {frame - 1}')
        #Image Difference
        if verbose == True:
          display(f'Attempting to use Image Difference for the {tracked_item} at frame {frame - 1}')
        motion_difference_boundary = [int(Torso_Size[0]/8), int(Torso_Size[0]/2), int(Torso_Size[0]/4), int(Torso_Size[0]/4)]
        if tracked_item == 'Left_BellGuard':
          boundary_box = create_boundary_box(expected_position, motion_difference_boundary, False)
          diff_position = motion_difference_tracking(frame, 'Left', [x_min, x_max, y_min, y_max], capture_width, capture_height, 1, 1, 2, orig_img_worpt_starting_list)
          if diff_position == 'None':
            diff_position = motion_difference_tracking(frame, 'Left', [x_min, x_max, y_min, y_max], capture_width, capture_height, 2, 1, 2, orig_img_worpt_starting_list)
        else:
          #Right Bellguard is assumed
          boundary_box = create_boundary_box(expected_position, motion_difference_boundary, True)
          diff_position = motion_difference_tracking(frame, 'Right', [x_min, x_max, y_min, y_max], capture_width, capture_height, 1, 1, 2, orig_img_worpt_starting_list)
          if diff_position == 'None':
            diff_position = motion_difference_tracking(frame, 'Right', [x_min, x_max, y_min, y_max], capture_width, capture_height, 2, 1, 2, orig_img_worpt_starting_list)
        box_test = boundary_box_test(diff_position, motion_difference_boundary)
        if box_test == True and diff_position != 'None':
          position = diff_position
          using_difference_images = True
        else:
          #Expected Position
          position = expected_position
          using_expected = True
        if verbose == True:
          display(f'The position for motion difference frame {frame - 1} is ({position})')
          display(f'The motion_difference_boundary test limits are {motion_difference_boundary} for frame {frame - 1}.')

      # Designed to catch an engarde position that is outside the tracking box
      if frame < (engarde_length + 3) and position == twice_previous_position:
        position = positions[0]

      #Sets Certainty Box
      # if (using_human_pose == True and fencer_data_side[0][2] > wrist_conf_high) or (using_position == True):
      if using_position == True:
        certainty = 0
        if verbose == True:
          display(f'Certainty set to zero for frame {frame - 1} for the {tracked_item}.')
      else:
        certainty = certainty + 1

    else:
      position = positions[0]

  # Multiple bounding boxes within the tracking box
  elif (len(positions)) > 1:
    if verbose == True:
      display(f'Multiple Bounding Boxes Detected for the {tracked_item} at frame {frame - 1}')
    # One set of conditions is used for Bell_Guards and another for all else
    if tracked_item == 'Left_BellGuard' or tracked_item == 'Right_BellGuard':
      if positions[0][2] > bellguard_confidence_high:
        if verbose == True:
          display(f'Using Multiple Box Determination for the {tracked_item} at frame {frame - 1}.')
        human_pose_boundary = [int(Torso_Size[0]*3/4), int(Torso_Size[0]), int(Torso_Size[0]/2), int(Torso_Size[0]/2)]
        position = multiple_box_determination(expected_position, positions, [human_pose_boundary[0], human_pose_boundary[1]], bellguard_confidence, horiz_flip)
        using_position = True
      else:
        position = expected_position
        using_expected = True
        if verbose == True:
          display(f'The Human Pose Box Test failed for the {tracked_item} at frame {frame - 1}, using expected position.')
      within_distance_from_expected = []
      for i in range(len(positions)):
        expected_box = [int(Torso_Size[0]/2*(1+bell_certainty/4)), int(Torso_Size[0]*(1+bell_certainty/4)), int(Torso_Size[0]/2*(1+bell_certainty/4)), int(Torso_Size[0]/6)]
        if tracked_item == 'Left_BellGuard':
          boundary_box = create_boundary_box(expected_position, expected_box, False)
        else:
          boundary_box = create_boundary_box(expected_position, expected_box, True)
        box_test = boundary_box_test(positions[i], boundary_box)
        if box_test:
          within_distance_from_expected.append(positions[i])

        # Uses the most confident, i.e. the first position in the list
        if len(within_distance_from_expected) > 0:
          position_boundary = [int(Torso_Size[0]/4), int(Torso_Size[0]/2), int(Torso_Size[0]/2), int(Torso_Size[0]/2)]
          position = multiple_box_determination(expected_position, positions, [position_boundary[0], position_boundary[1]], bellguard_confidence, horiz_flip)
          certainty = 0
          using_position = True
        else:
          # If the length of within_distance_from_expected is zero
          if verbose == True:
            display(f'Error occured finding a position within the required distance and the {tracked_item} set to expected position at frame {frame - 1}.')
            display(f'The expected position is {expected_position}, while the expected box is {expected_box}.')
          position = [(x_pos + x_speed),(y_pos + y_speed)]
          using_expected = True

      #Sets Certainty Box
      # if (using_human_pose == True and fencer_data_side[0][2] > wrist_conf_high) or (using_position == True):
      if using_position == True:
        if verbose == True:
          display(f'Confidence for the {tracked_item} is High so the certainty is set to zero.')
        certainty = 0
      else:
        if verbose == True:
          display(f'Confidence for the {tracked_item} is Low so the certainty is incremented higher.')
        certainty = certainty + 1

    elif tracked_item == 'Left_Foot' or tracked_item == 'Right_Foot':
      # Max and Min are based on the first value of the set therefore in this case max and min refer to the x position
      if horiz_flip == False:
        position = max(positions)
      else:
        position = min(positions)
      certainty = 0

    # If the tracked item is not a bell_guard
    else:
      # Uses the most confident position within the tracking box
      position = positions[0]
      # Sets Certainty for Torso and Box back to Zero if detected.
      certainty = 0

  if tracked_item == 'Left_BellGuard' or tracked_item == 'Right_BellGuard':
    if verbose == True:
      display(f'The position of the {tracked_item} at frame {frame - 1} is {position}.')

  return (position, certainty, [x_min, x_max, y_min, y_max])

In [ ]:
#@title Weight_Average_List
def weight_average_list(List):
  # Finds the Weight Average of a List

  # Prevents division by zero
  try:
    value_sum = 0
    value_weight = 0
    for i in range(len(List)):
      value_sum = value_sum + List[i][0] * List[i][1]
      value_weight = value_weight + List[i][1]
    weighted_average = value_sum/value_weight
  except:
    weighted_average = 0

  return (weighted_average)

In [ ]:
#@title Average_List
def average_list(List):
  # Finds the Average of a List
  try:
    average = sum(List) / len(List)
  except:
    average = 0
  return (average)

In [ ]:
def average_list_without_null(List):
  # Takes a List of x,y values and null values. Removes the null values and returns the average x and y values.

  List_temp = []
  for i in range(len(List)):
    if List[i] != []:
      List_temp.append(List[i])
    else:
      pass
  
  x_sum = 0
  y_sum = 0

  # display(f'List_Temp is {List_temp}')

  for j in range(len(List_temp)):
    x_sum = x_sum + List_temp[j][0]
    y_sum = y_sum + List_temp[j][1]

  x_average = int(x_sum/len(List_temp))
  y_average = int(y_sum/len(List_temp))

  average = [x_average, y_average]

  return (average)

In [ ]:
#@title Color_Tester
def color_tester(box, frame):
  #Takes a given box and tests for a specific color range

  path = r'/content/Mask_RCNN/videos/save/'
  file_name = str(frame) + '.jpg'
  name = os.path.join(path, file_name)
  img = cv2.imread(name)

  if verbose == True:
    display(f'The file names to be color tested is {name}.')
  # box[0] are the coordinates ([y1,x1,y2,x2]), box[1] is confidence and box[2] is object
  # Tests if Bellguard is the correct color
  if box[2] == 1:
    blue_range = [50, 150]
    green_range = [50, 150]
    red_range = [50, 160]
    max_delta = 25
  elif box[2] == 3:
    blue_range = [60, 150]
    green_range = [60, 150]
    red_range = [60, 160]
    max_delta = 30
  else:
    if verbose == True:
      display(f'The object to test does not have a color profile.')

  # OpenCV uses Blue, Green, Red order
  b, g, r = 0, 0, 0

  width = (box[0][3]-box[0][1])
  height = (box[0][2]-box[0][0])

  #i is the x value of the image
  for i in range(width):
    #j is y value of the image
    for j in range(height):
      #color channel of the image [B,G,R]
      #image, img, is of format [y,x] 
      b = b + img[box[0][0] + j, box[0][1] + i, 0]
      g = g + img[box[0][0] + j, box[0][1] + i, 1]
      r = r + img[box[0][0] + j, box[0][1] + i, 2]

  # Finds the Color Averages
  b_average = int(b/(width*height))
  g_average = int(g/(width*height))
  r_average = int(r/(width*height))

  # Finds maximum differences between colors
  max_1 = abs(b_average - g_average)
  max_2 = abs(b_average - r_average)
  max_3 = abs(g_average - r_average)
  max_delta = max(max_1, max_2, max_3)

  if test_result == False:
    if verbose == True:
      display(f'The Color Test Result Failed for object {box[2]}.')

  return (test_result)

In [ ]:
#@title Symmetry_Test
def symmetry_test(width, height, left_x, left_y, right_x, right_y):

  # Tests the potential left and right positions for left/right symmetry and removes outlier points
  display(f'Commencing Symmetry Test...')

  # Sets how large the allowable band is with respect to height or width
  band_width_ratio_x = 8
  band_width_ratio_y = 8

  all_positions_x = left_x + right_x
  all_positions_y = left_y +right_y
  if len(all_positions_x) != len(all_positions_y):
    display(f'ERROR...The length of the x and y positions are different.')


  # Keeps track of which positions are most in line with the other positions
  # Finds the X Band
  x_distances_from_center = []
  x_distances_from_other_points_score = []
  for i in range(len(all_positions_x)):
    #Determines the x_min band for each position by distance from center
    x_distances_from_center.append(abs(int((width/2)-all_positions_x[i])))
  #Creates an iterator that determines which x_point is close to the most other points and finds its index
  for j in range(len(x_distances_from_center)):
    score = 0
    for k in range(len(x_distances_from_center) - 1):
      if abs(x_distances_from_center[j] - x_distances_from_center[k+1]) < width/band_width_ratio_x:
        score = score + 1
      else:
        pass
    x_distances_from_other_points_score.append(score)
  x_index_band = x_distances_from_other_points_score.index(max(x_distances_from_other_points_score))

  x_min = abs(int(all_positions_x[x_index_band] - width/band_width_ratio_x))
  x_max = abs(int(all_positions_x[x_index_band] + width/band_width_ratio_x))

  # Finds the Y Band
  y_distances_from_center = []
  y_distances_from_other_points_score = []
  for i in range(len(all_positions_y)):
    y_distances_from_center.append(abs(int((height/2)-all_positions_y[i])))
  for j in range(len(y_distances_from_center)):
    score = 0
    for k in range(len(y_distances_from_center) - 1):
      if abs(y_distances_from_center[j] - y_distances_from_center[k+1]) < width/band_width_ratio_y:
        score = score + 1
      else:
        pass
    y_distances_from_other_points_score.append(score)
  y_index_band = y_distances_from_other_points_score.index(max(y_distances_from_other_points_score))

  y_min = abs(int(all_positions_y[y_index_band] - width/band_width_ratio_y))
  y_max = abs(int(all_positions_y[y_index_band] + width/band_width_ratio_y))

  # Cycles through the positions and keeps values that are in the horizontal x band
  positionsx_temp = []
  positionsy_temp = []

  if verbose == True:
    display(f'The x_min/max is {x_min}/{x_max}, the band width is {width/band_width_ratio_x} and the center is {width/2}.')

  for i in range(len(all_positions_x)):
    if ((all_positions_x[i] < (width/2 - x_min)) and (all_positions_x[i] > (width/2 - x_max))) or ((all_positions_x[i] < (width/2 + x_max)) and (all_positions_x[i] > (width/2 + x_min))):
      positionsx_temp.append(all_positions_x[i])
      positionsy_temp.append(all_positions_y[i])
    else:
      pass

  # Replaces the all position x and y lists with the temp list limited by the bands
  all_positions_x = positionsx_temp
  all_positions_y = positionsy_temp

  #Cycles through the positions and keeps values that are in the vertical y band
  positionsx_temp = []
  positionsy_temp = []

  if verbose == True:
    display(f'The y_min/max is {y_min}/{y_max}, the band width is {height/band_width_ratio_y} and the center is {height/2}.')

  for i in range(len(all_positions_y)):
    if ((all_positions_y[i] > (y_min)) and (all_positions_y[i] < (y_max))):
      positionsx_temp.append(all_positions_x[i])
      positionsy_temp.append(all_positions_y[i])
    else:
      pass

  # Replaces the all position x and y lists with the temp list limited by the bands
  all_positions_x = positionsx_temp
  all_positions_y = positionsy_temp

  if verbose == True:
    display(f'There were originaly {len(left_x) + len(right_x)} values and {len(all_positions_x) - (len(left_x) + len(right_x))} were removed.')

  # Returns the x and y values to left and right positions
  ret_left_x, ret_left_y, ret_right_x, ret_right_y = [],[],[],[]

  
  for i in range(len(all_positions_x)):
    # Tests if the x value is on the left or right side
    if all_positions_x[i] < width/2:
      ret_left_x.append(all_positions_x[i])
      ret_left_y.append(all_positions_y[i])
    else:
      ret_right_x.append(all_positions_x[i])
      ret_right_y.append(all_positions_y[i])
  # Prevents an off center camera from removing all engarde points
  if (len(ret_left_x) == 0) or (len(ret_left_y) == 0) or (len(ret_right_x) == 0) or (len(ret_right_y) == 0):
    ret_left_x = left_x
    ret_left_y = left_y
    ret_right_x = right_x
    ret_right_y = right_y

  return (ret_left_x, ret_left_y, ret_right_x, ret_right_y)

In [ ]:
#@title List_Threshold_Test
def list_threshold_test(threshold, list_to_test):
  #Determines if a list meets a minimum threshold
  threshold_met = False

  for k in range(len(list_to_test)):
    if list_to_test[k][1] > threshold:
      threshold_met = True
      break
    else:
      pass

  return (threshold_met)

In [ ]:
#@title Multiple_Box_Determination
def multiple_box_determination(expected_position, positions, x_boundaries, min_conf, horiz_flip):

  confidence_weighting = .9

  delta_x_forward = x_boundaries[1]
  delta_x_backward = x_boundaries[0]

  if horiz_flip == True:
    delta_temp = delta_x_forward
    delta_x_forward = delta_x_backward
    delta_x_backward = delta_temp

  position_ratings = []

  if verbose == True:
    display(f'There are {len(positions)} positions available.')
    display(f'The positions are:')
    display(positions)  

  for i in range(len(positions)):
    delta_position = positions[i][0] - expected_position[0]
    if verbose == True:
      display(f'The positions{i}[0] is {positions[i][0]} and the expected_position[0] is {expected_position[0]} therefore delta position is {delta_position}.')
    if delta_position > 0:
      if verbose == True:
        display(f'Position {i} is forward of the expected position.')
      position_ratings.append(abs((delta_position/delta_x_forward)*(1-positions[i][2])**confidence_weighting))
      if verbose == True:
        display(f'delta_position is {delta_position}.')
        display(f'delta_x_forward is {delta_x_forward}.')
        display(f'positions[i][2] is {positions[i][2]}.')
    else:
      if verbose == True:
        display(f'Position {i} is behind the expected position.')
      position_ratings.append(abs((delta_position/delta_x_backward)*(1-positions[i][2])**confidence_weighting))
      if verbose == True:
        display(f'delta_position is {delta_position}.')
        display(f'delta_x_backward is {delta_x_backward}.')
        display(f'positions[i][2] is {positions[i][2]}.')

  if verbose == True:
    display(position_ratings)

  position = positions[position_ratings.index(min(position_ratings))]

  return (position)

In [ ]:
#@title Boundary_Box_Overlap
def boundary_box_overlap(box1, box2):
  #Finds the overlap of two boxes assume (x_min, x_max, y_min, y_max)
  
  box_overlap = [max(box1[0], box2[0]), min(box1[1], box2[1]), max(box1[2], box2[2]), min(box1[3], box2[3])]

  return (box_overlap)

In [ ]:
#@title Create_Boundary_Box
def create_boundary_box(center, padding, horiz_flip):
  # Creates a Boundary Box based on Center Padding and if the Left and Right Boundaries should be flipped.
  # Center is [x,y]
  # Padding is [Behind, Front, Top, Bottom]
  # horiz_flip is True or False

  if horiz_flip == False:
    left = center[0] - padding[0]
    right = center[0] + padding[1]
  elif horiz_flip == True:
    left = center[0] - padding[1]
    right = center[0] + padding[0]
  else:
    if verbose == True:
      display(f'ERROR Horiz Flip not True or False.')

  top = center[1] - padding[2]
  bottom = center[1] + padding[3]

  return ([left, right, top, bottom])

In [ ]:
#@title Boundary_Box_Test
def boundary_box_test(test_point, boundary):
  # Tests if a give point is in a Boundary Box.
  #Format Test_Point is of the form (x,y)
  #Format Boundary is of the form (x_min, x_max, y_min, y_max)
  #Format Boundary is of the form (behind the fencer, in front of the fencer, above the fencer, below the fencer)

  if verbose == True:
    display(test_point)
    display(boundary)

  if test_point != 'None':
    if test_point[0] > boundary[0] and test_point[0] < boundary[1] and test_point[1] > boundary[2] and test_point[1] < boundary[3]:
      box_test = True
    else:
      box_test = False
  else:
    box_test = False

  return (box_test)

In [ ]:
#@title Engarde_Failure_Test
def engarde_failure_test(bbox, bellguard_confidence, x_max, y_max, side):
  # Tests for reasons the engarde positioning failed to detect a BellGuard

  if verbose == True:
    display(f'The {side} engarde position failed due to...')

  if side == 'Left':
    oppside = 'Right'
    k = 0
  else:
    oppside = 'Left'
    k = 1


  for j in range(len(bbox)):
    if bbox[j][1] < bellguard_confidence:
      if verbose == True:
        display(f'The confidence in the {side} bellguard is too low at {bellguard_confidence}.')
    else: 
      pass
    if side == 'Left':
      if bbox[j][k] > x_max:
        if verbose == True:
          display(f'The {side} bellguard was too far {oppside} at {bbox[j][0]} while the maximum is {x_max}.')
      else:
        pass
    else:
      if verbose == True:
        display(f'bbox at this point is: {bbox}. J is {j} and k is {k}.')
        display(bbox[j])
        display(bbox[j][k])
      if bbox[j][k] < x_max:
        if verbose == True:
          display(f'The {side} bellguard was too far {oppside} at {bbox[j][0]} while the maximum is {x_max}.')
    if bbox[j][k] > y_max:
      if verbose == True:
        display(f'The {side} bellguard was too low at {bbox[j][0]} while the maximum allowed is {y_max}.')
    else:
      pass

  return

In [ ]:
#@title Torso_Failure_Test
def torso_failure_test(bbox, capture_width, capture_height, y_average, Bell_Guard_Size_average, side, frame_count, min_torso_confidence):
  # Tests for reasons the engarde positioning failed to detect a Torso
  # Is tested at finding tracking boxes

  if verbose == True:
    display(f'The {side} Torso failed due to...') 
  for j in range(len(bbox)):
    if bbox[j][1] > min_torso_confidence:
      pass
    else:
      if verbose == True:
        display(f'The confidence is of the box is too low at only {int(bbox[j][1]*100)}% at frame {frame_count}.')
    if bbox[j][0][2] > y_average:
      pass
    else:
      if verbose == True:
        display(f'The Torso was not lower than the Bell Guard with a lower height of {bbox[j][0][2]} with a max value of {y_average} at frame {frame_count}.')
    if bbox[j][0][2] < (y_average + 3*Bell_Guard_Size_average[1]):
      pass
    else:
      if verbose == True:
        display(f'The bottom of the torso box was too low at {bbox[j][0][2]} with a max value of {int(y_average + 3*Bell_Guard_Size_average[1])} at frame {frame_count}.')

  if verbose == True:
    display(f'y_average is {y_average}.')
    display(f'Bell_Guard_Size_average[1] is {Bell_Guard_Size_average[1]}.')

  return

In [ ]:
#@title Torso_Position_Failure_Test
def torso_position_failure_test(bbox, engarde_length, x_min_torso, x_max_torso, y_min_torso, y_max_torso, y_average, side, frame_count):
  # Tests for reasons the engarde positioning failed to detect a Torso
  # Is tested at torso positions

  confidence = min_torso_confidence

  if verbose == True:
    display(f'Analyzing the Torso Position Failure at frame {frame_count} for the {side} side...')
  count = 0
  
  for k in range(len(bbox)):

    if bbox[k][2] == 3 and bbox[k][1] > confidence:
      count = count + 1
  if verbose == True:
    display(f'There are {len(bbox)} ROIs, {count} of them are Torsos with greater than {confidence}%.')

  for j in range(len(bbox)):
    y_center = int((bbox[j][0][0] + bbox[j][0][2])/2)
    x_center = int((bbox[j][0][1] + bbox[j][0][3])/2)
    if bbox[j][2] == 3 and bbox[j][1] > confidence:
      if x_center > x_min_torso:
        pass
      else:
        if verbose == True:
          display(f'The Torso center at {x_center} is to the Left of the Box side at {x_min_torso} at frame {frame_count}.')
      if x_center < x_max_torso:
        pass
      else:
        if verbose == True:
          display(f'The Torso center at {x_center} is to the Right of the Box side at {x_max_torso} at frame {frame_count}.')
      if y_center > y_min_torso:
        pass
      else:
        if verbose == True:
          display(f'The Torso center at {y_center} is Above the Box at {y_min_torso} at frame {frame_count}.')
      if y_center < y_max_torso:
        pass
      else:
        if verbose == True:
          display(f'The Torso center at {y_center} is Below the Box at {y_max_torso} at frame {frame_count}.')
      if bbox[j][0][2] > y_average:
        pass
      else:
        if verbose == True:
          display(f'The Torso center is Below the Bell Guard at frame {frame_count}.')
      if bbox[j][2] == 3:
        pass
      else:
        if verbose == True:
          display(f'The Torso is not labelled as a Torso at frame {frame_count}.')
    else:
      pass

  return

In [ ]:
#@title Engarde_Position
def engarde_position(bbox, capture_width, capture_height, engarde_length, frame_count, save_image_list):
  #Finds the initial positions to start tracking
  #Format of bbox[frame][roi], ([y1,x1,y2,x2], percent certainty, type)

  # Initializes the Bell Guard Positions
  # Position format [x,y]
  # Size format [[Width],[Height]]
  Left_Position = []
  Right_Position = []
  Bell_Guard_Size = [[],[]]
  Scoring_Box_Position = []
  Scoring_Box_Size = [[],[]]
  Left_Torso_Position = []
  Left_Torso_Size = [[],[]]
  Right_Torso_Position = []
  Right_Torso_Size = [[],[]]
  Left_Foot_Position = []
  Right_Foot_Position = []
  Foot_Size = [[],[]]
  All_Bell_Guard_Positions = []

  # sum_of_boxes is used to average the Left (x,y)(0), Right (x,y)(1), ScoreBox (x,y)(2), Left_Torso (x,y)(3), Right_Torso(x,y)(4),
  # Left_Foot(x,y)(5), Right_Foot(x,y),(6) values
  sum_of_boxes = [[[],[]],[[],[]],[[],[]],[[],[]],[[],[]],[[],[]],[[],[]]]

  # j represents the rois(specific bounding box) within the frame sorted by confidence score
  for j in range(len(bbox)):
    # The percent confidence for each region of interest (roi) is [i][j][1]
    # This uses the minimum value of the bbox (top-left) to determine Left, Right, Scorebox
    # The Bellguards must be centered within the frame, classified as Bellguards with a minimum confidence and have the correct color saturation
    # Adds values to the Left engarde box
    if (bbox[j][1] > bellguard_confidence and bbox[j][0][1] < int(capture_width*2/5) and bbox[j][0][0] < int(capture_height*3/4) and bbox[j][0][0] > int(capture_height*1/4) and bbox[j][2] == 1):
      test_result = saturation_test(bbox[j], frame_count, save_image_list)
      if verbose == True:
        display(f'The result of the saturation test for the Left Engarde Position is {test_result} at frame {frame_count}.')
      if test_result == True:
        #Appends x value:
        sum_of_boxes[0][0].append([bbox[j][0][1], bbox[j][1]])
        #Appends y value:
        sum_of_boxes[0][1].append([bbox[j][0][0], bbox[j][1]])
        #Appends x width value:
        Bell_Guard_Size[0].append(bbox[j][0][3] - bbox[j][0][1])
        #Appends y width value:
        Bell_Guard_Size[1].append(bbox[j][0][2] - bbox[j][0][0])
    #Adds values to the Right engarde box
    elif (bbox[j][1] > bellguard_confidence and bbox[j][0][1] > int(capture_width*3/5) and bbox[j][0][0] < int(capture_height*3/4) and bbox[j][0][0] > int(capture_height*1/4) and bbox[j][2] == 1):
      # test_result = color_tester(bbox[i][j], i)
      test_result = saturation_test(bbox[j], frame_count, save_image_list)
      if verbose == True:
        display(f'The result of the saturation test for the Right Engarde Position is {test_result} at frame {frame_count}.')
      if test_result == True:
        #Appends x value:
        sum_of_boxes[1][0].append([bbox[j][0][1], bbox[j][1]])
        #Appends y value:
        sum_of_boxes[1][1].append([bbox[j][0][0], bbox[j][1]])
        #Appends x width value:
        Bell_Guard_Size[0].append(bbox[j][0][3] - bbox[j][0][1])
        #Appends y width value:
        Bell_Guard_Size[1].append(bbox[j][0][2] - bbox[j][0][0])
    #Adds values to the ScoreBox Position
    elif (bbox[j][1] > 0.50 and bbox[j][0][1] > int(capture_width/3) and bbox[j][0][1] < int(capture_width*(2/3)) and bbox[j][2] == 2):
      #Appends x value:
      sum_of_boxes[2][0].append([bbox[j][0][1], bbox[j][1]])
      #Appends y value:
      sum_of_boxes[2][1].append([bbox[j][0][0], bbox[j][1]])  
      #Appends x width value:
      Scoring_Box_Size[0].append(bbox[j][0][3] - bbox[j][0][1])
      #Appends y width value:
      Scoring_Box_Size[1].append(bbox[j][0][2] - bbox[j][0][0])
    # Adds values to the Left Foot Position
    elif (bbox[j][1] > foot_confidence and bbox[j][0][1] < int(capture_width*2/5) and bbox[j][0][0] > int(capture_height*2/4) and bbox[j][2] == 4):
      sum_of_boxes[5][0].append([bbox[j][0][1], bbox[j][1]])
      sum_of_boxes[5][1].append([bbox[j][0][0], bbox[j][1]])  
      Foot_Size[0].append(bbox[j][0][3] - bbox[j][0][1])
      Foot_Size[1].append(bbox[j][0][2] - bbox[j][0][0])
    # Adds values to the Right Foot Position
    elif (bbox[j][1] > foot_confidence and bbox[j][0][1] > int(capture_width*2/5) and bbox[j][0][0] > int(capture_height*2/4) and bbox[j][2] == 4):
      sum_of_boxes[6][0].append([bbox[j][0][1], bbox[j][1]])
      sum_of_boxes[6][1].append([bbox[j][0][0], bbox[j][1]])  
      Foot_Size[0].append(bbox[j][0][3] - bbox[j][0][1])
      Foot_Size[1].append(bbox[j][0][2] - bbox[j][0][0])
    else:
      pass
    
    if bbox[j][2] == 1:
      All_Bell_Guard_Positions.append([int((bbox[j][0][3] + bbox[j][0][1])/2), int((bbox[j][0][2] + bbox[j][0][0])/2)])

  if verbose == True:
    display(f'The Left Foot Sum of Boxes at frame {frame_count} is:')
    display(sum_of_boxes[5])

  try:
    # Tests for cause of Left Engarde Position Failure
    if len(sum_of_boxes[0][0]) == 0:
      engarde_failure_test(bbox[j], bellguard_confidence, int(capture_width*2/5), int(capture_height*2/3), 'Left')
    # Tests for cause of Right Engarde Position Failure
    if len(sum_of_boxes[1][0]) == 0:
      engarde_failure_test(bbox[j], bellguard_confidence, int(capture_width*3/5), int(capture_height*3/4), 'Right')
  except:
    if verbose == True:
      display(f'There was an error in the engarde failure test and it was skipped.')

  # Finds the center points
  x_average_left = weight_average_list(sum_of_boxes[0][0])
  y_average_left = weight_average_list(sum_of_boxes[0][1])
  x_average_right = weight_average_list(sum_of_boxes[1][0])
  y_average_right = weight_average_list(sum_of_boxes[1][1])
  x_average_scorebox = weight_average_list(sum_of_boxes[2][0])
  y_average_scorebox = weight_average_list(sum_of_boxes[2][1])
  x_average_left_foot = weight_average_list(sum_of_boxes[5][0])
  y_average_left_foot = weight_average_list(sum_of_boxes[5][1])
  x_average_right_foot = weight_average_list(sum_of_boxes[6][0])
  y_average_right_foot = weight_average_list(sum_of_boxes[6][1])

  # Prevents a failure to detect the bellguard from failing to detect the torso
  # If the bellguard is unusually high or low then it is set to the height of the opposing BellGuard
  if (y_average_left < capture_height/5) or (y_average_left > capture_height*4/5):
    if verbose == True:
      display(f'The y_average_left was too high or low and was set to y_average_right.')
    y_average_left = y_average_right
  if (y_average_right < capture_height/5) or (y_average_right > capture_height*4/5):
    if verbose == True:
      display(f'The y_average_right was too high or low and was set to y_average_left.')
    y_average_right = y_average_left

  if verbose == True:
    display(f'The average left position is ({x_average_left},{y_average_left}).')
    display(f'The average right position is ({x_average_right},{y_average_right}).')

  # Bell_Guard_Size_average [Width, Height]
  Bell_Guard_Size_average = []
  # Appends the average scoring box width
  Bell_Guard_Size_average.append(average_list(Bell_Guard_Size[0]))
  # Appends the average scoring box height
  Bell_Guard_Size_average.append(average_list(Bell_Guard_Size[1]))

  # Finds the Torso Position After the Bell_Guard Position because the Bell_Guard is used as a constraint
  # j represents the rois(specific bounding box) within the frame sorted by confidence score
  for j in range(len(bbox)):
    # Adds values to the Left_Torso Position, similar requirements to Left guard
    # Minimum Torso confidence, on the left half of the screen, bottom of the box is below the bellguard, but also above 3 three times the bellguard height and is labeled torso
    if (bbox[j][1] > min_torso_confidence and bbox[j][0][1] < int(capture_width/2) and bbox[j][0][2] > (y_average_left - Bell_Guard_Size_average[1] * 2) \
        and bbox[j][0][2] < (y_average_left + 3*Bell_Guard_Size_average[1]) and bbox[j][2] == 3):
      test_result = True
      if test_result == True:
        # Appends x value:
        sum_of_boxes[3][0].append(bbox[j][0][1])
        #Appends y value:
        sum_of_boxes[3][1].append(bbox[j][0][0])
        #Appends x width value:
        Left_Torso_Size[0].append(bbox[j][0][3] - bbox[j][0][1])
        #Appends y width value:
        Left_Torso_Size[1].append(bbox[j][0][2] - bbox[j][0][0])
      else:
        if verbose == True:
          display(f'The saturation test failed at frame {frame_count}.')
        else:
          pass
    # Adds values to the Right_Torso Position, similar requirements to Right guard
    # display(f'y_average_right-Bell_Guard_Size[1] * 2 is {(y_average_right-Bell_Guard_Size[1]*2)}.')

    if Bell_Guard_Size[1] == []:
      if verbose == True:
        display(f'The Bell Guard Height was not defined so it is set to a default of zero at frame {frame_count}.')
      Bell_Guard_Size[1] = 0

    if verbose == True:
      display(f'bbox[j][1] is {bbox[j][1]}.')
      display(f'bbox[j][0][1] is {bbox[j][0][1]}.')
      display(f'bbox[j][0][2] is {bbox[j][0][2]}.')
      display(f'y_average_right is {y_average_right}')
      display(f'Bell_Guard_Size[1] is {Bell_Guard_Size[1]}')
      display(f'(y_average_right-2*Bell_Guard_Size[1]) is {(y_average_right-2*Bell_Guard_Size[1])}.')
      display(f'bbox[j][1] is {bbox[j][1]}.')

    if (bbox[j][1] > min_torso_confidence and bbox[j][0][1] > int(capture_width/2) and bbox[j][0][2] > (y_average_right-2*Bell_Guard_Size_average[1]) and bbox[j][0][2] < (y_average_right + 3*Bell_Guard_Size_average[1]) and bbox[j][2] == 3):
      
      #Appends x value:
      sum_of_boxes[4][0].append(bbox[j][0][1])
      #Appends y value:
      sum_of_boxes[4][1].append(bbox[j][0][0])
      #Appends x width value:
      Right_Torso_Size[0].append(bbox[j][0][3] - bbox[j][0][1])
      #Appends y width value:
      Right_Torso_Size[1].append(bbox[j][0][2] - bbox[j][0][0])

  # Checks for a Failure to Detect the Left Torso
  if len(sum_of_boxes[3][0]) == 0:
    torso_failure_test(bbox, capture_width, capture_height, y_average_left, Bell_Guard_Size_average, 'Left', frame_count, min_torso_confidence)
  if verbose == True:
    display(f'Prior to torso failure test for right torso the y_average_left is {y_average_left}.')

  # Checks for a Failure to Detect the Right Torso
  if len(sum_of_boxes[4][0]) == 0:
    torso_failure_test(bbox, capture_width, capture_height, y_average_right, Bell_Guard_Size_average, 'Right', frame_count, min_torso_confidence) 
  if verbose == True:
    display(f'Prior to torso failure test for left torso the y_average_right is {y_average_right}.')

  #Finds the top left corner then moves the average point to the center
  x_average_left_torso = average_list(sum_of_boxes[3][0]) + average_list(Left_Torso_Size[0])/2
  y_average_left_torso = average_list(sum_of_boxes[3][1]) + average_list(Left_Torso_Size[1])/2
  x_average_right_torso = average_list(sum_of_boxes[4][0]) + average_list(Right_Torso_Size[0])/2
  y_average_right_torso = average_list(sum_of_boxes[4][1]) + average_list(Right_Torso_Size[1])/2

  if verbose == True:
    display(f'The average left engarde position is:({x_average_left},{y_average_left})')
    display(f'The average right engarde position is:({x_average_right},{y_average_right})')
    display(f'The average left torso is:({int(x_average_left_torso)},{int(y_average_left_torso)})')
    display(f'The average right torso is:({int(x_average_right_torso)},{int(y_average_right_torso)})')

  # scoring_box_size_average [Width, Height]
  scoring_box_size_average = []
  # Appends the average scoring box width
  scoring_box_size_average.append(average_list(Scoring_Box_Size[0]))
  # Appends the average scoring box height
  scoring_box_size_average.append(average_list(Scoring_Box_Size[1]))

  # left_torso_size_average [Width, Height]
  left_torso_size_average = []
  # Appends the average scoring box width
  left_torso_size_average.append(average_list(Left_Torso_Size[0]))
  # Appends the average scoring box height
  left_torso_size_average.append(average_list(Left_Torso_Size[1]))

  # right_torso_size_average [Width, Height]
  right_torso_size_average = []
  # Appends the average scoring box width
  right_torso_size_average.append(average_list(Right_Torso_Size[0]))
  # Appends the average scoring box height
  right_torso_size_average.append(average_list(Right_Torso_Size[1]))

  #Creates Padding for the EnGarde Tracking Box
  engarde_box_padding = int(capture_width/15)
  torso_padding = int(capture_width/20)
  foot_padding = int(capture_width/15)

  x_min_engardeL = int(x_average_left - engarde_box_padding)
  x_max_engardeL = int(x_average_left + engarde_box_padding)
  y_min_engardeL = int(y_average_left - engarde_box_padding)
  y_max_engardeL = int(y_average_left + engarde_box_padding)

  x_min_engardeR = int(x_average_right - engarde_box_padding)
  x_max_engardeR = int(x_average_right + engarde_box_padding)
  y_min_engardeR = int(y_average_right - engarde_box_padding)
  y_max_engardeR = int(y_average_right + engarde_box_padding)

  x_min_engardeScore = int(x_average_scorebox - engarde_box_padding)
  x_max_engardeScore = int(x_average_scorebox + engarde_box_padding)
  y_min_engardeScore = int(y_average_scorebox - engarde_box_padding)
  y_max_engardeScore = int(y_average_scorebox + engarde_box_padding)

  x_min_torsoL = int(x_average_left_torso - torso_padding)
  x_max_torsoL = int(x_average_left_torso + torso_padding)
  y_min_torsoL = int(y_average_left_torso - torso_padding*3/2)
  y_max_torsoL = int(y_average_left_torso + torso_padding*3/2)

  x_min_torsoR = int(x_average_right_torso - torso_padding)
  x_max_torsoR = int(x_average_right_torso + torso_padding)
  y_min_torsoR = int(y_average_right_torso - torso_padding*3/2)
  y_max_torsoR = int(y_average_right_torso + torso_padding*3/2)

  x_min_footL = int(x_average_left_foot)
  x_max_footL = int(x_average_left_foot + foot_padding*2)
  y_min_footL = int(y_average_left_foot - foot_padding)
  y_max_footL = int(y_average_left_foot + foot_padding)

  if verbose == True:
    display(f'The Left Foot xmin,xmax,ymin,ymax are {x_min_footL},{x_max_footL},{y_min_footL},{y_max_footL} at frame {frame_count}.')

  # x Foot Padding is large because both feet are detected and differentiated in the subsequent steps
  x_min_footR = int(x_average_right_foot - foot_padding*2)
  x_max_footR = int(x_average_right_foot)
  y_min_footR = int(y_average_right_foot - foot_padding)
  y_max_footR = int(y_average_right_foot + foot_padding)

  #Iterates through the first engarde_length frames and checks if there are rois in the expected engarde position
  for j in range(len(bbox)):
    y_center = int((bbox[j][0][0] + bbox[j][0][2])/2)
    x_center = int((bbox[j][0][1] + bbox[j][0][3])/2)
    # Checks for rois in the Left Engarde Position
    if (x_center > x_min_engardeL and x_center < x_max_engardeL and y_center > y_min_engardeL and y_center < y_max_engardeL and bbox[j][2] == 1):
      # display(f'The roi is in the left en garde position')
      Left_Position.append([x_center, y_center])
    # Checks for rois in the Right Engarde Position
    if (x_center > x_min_engardeR and x_center < x_max_engardeR and y_center > y_min_engardeR and y_center < y_max_engardeR and bbox[j][2] == 1):
      # display(f'The roi is in the right en garde position')
      Right_Position.append([x_center, y_center])
    # Checks for rois in the Scoring Box Position
    if (x_center > x_min_engardeScore and x_center < x_max_engardeScore and y_center > y_min_engardeScore and y_center < y_max_engardeScore and bbox[j][2] == 2):
      Scoring_Box_Position.append([x_center, y_center])
    # Checks for rois in the Left Torso Position
    if (x_center > x_min_torsoL and x_center < x_max_torsoL and y_center > y_min_torsoL and y_center < y_max_torsoL and bbox[j][0][2] > y_average_left and bbox[j][2] == 3):
      Left_Torso_Position.append([x_center, y_center])
    # Checks for rois in the Right Torso Position 
    if (x_center > x_min_torsoR and x_center < x_max_torsoR and y_center > y_min_torsoR and y_center < y_max_torsoR and bbox[j][0][2] > y_average_right and bbox[j][2] == 3):
      Right_Torso_Position.append([x_center, y_center])
    # Checks for rois in the Left Foot Position 
    if (x_center > x_min_footL and x_center < x_max_footL and y_center > y_min_footL and y_center < y_max_footL and bbox[j][0][2] > y_average_left and bbox[j][0][1] > x_average_left_torso and bbox[j][2] == 4):
      Left_Foot_Position.append([x_center, y_center])
    # Checks for rois in the Right Foot Position 
    if (x_center > x_min_footR and x_center < x_max_footR and y_center > y_min_footR and y_center < y_max_footR and bbox[j][0][2] > y_average_right and bbox[j][0][1] < x_average_right_torso and bbox[j][2] == 4):
      Right_Foot_Position.append([x_center, y_center])

    # Tracking Bounding Box has unused brackets for Left and Right Torso
    Tracking_Bounding_Boxes_Temp = [[],[],[],[],[],[],[]]

    Tracking_Bounding_Boxes_Temp[0].append(x_min_engardeL)
    Tracking_Bounding_Boxes_Temp[0].append(x_max_engardeL)
    Tracking_Bounding_Boxes_Temp[0].append(y_min_engardeL)
    Tracking_Bounding_Boxes_Temp[0].append(y_max_engardeL)

    Tracking_Bounding_Boxes_Temp[1].append(x_min_engardeR)
    Tracking_Bounding_Boxes_Temp[1].append(x_max_engardeR)
    Tracking_Bounding_Boxes_Temp[1].append(y_min_engardeR)
    Tracking_Bounding_Boxes_Temp[1].append(y_max_engardeR)

    Tracking_Bounding_Boxes_Temp[2].append(x_min_engardeScore)
    Tracking_Bounding_Boxes_Temp[2].append(x_max_engardeScore)
    Tracking_Bounding_Boxes_Temp[2].append(y_min_engardeScore)
    Tracking_Bounding_Boxes_Temp[2].append(y_max_engardeScore)

    Tracking_Bounding_Boxes_Temp[5].append(x_max_footL)
    Tracking_Bounding_Boxes_Temp[5].append(x_max_footL)
    Tracking_Bounding_Boxes_Temp[5].append(y_min_footL)
    Tracking_Bounding_Boxes_Temp[5].append(y_max_footL)

    Tracking_Bounding_Boxes_Temp[6].append(x_max_footR)
    Tracking_Bounding_Boxes_Temp[6].append(x_max_footR)
    Tracking_Bounding_Boxes_Temp[6].append(y_min_footR)
    Tracking_Bounding_Boxes_Temp[6].append(y_max_footR)

    Tracking_Bounding_Boxes = Tracking_Bounding_Boxes_Temp

  # Creates a Tracking Bounding Boxes Variable if there are no Bounding Box detections
  if len(bbox) == 0:
    Tracking_Bounding_Boxes = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[],[],[0,0,0,0],[0,0,0,0]]

  # Tests for why a Torso Position is not Found
  if (len(Left_Torso_Position) == 0):
    torso_position_failure_test(bbox, engarde_length, x_min_torsoL, x_max_torsoL, y_min_torsoL, y_max_torsoL, y_average_left, 'Left', frame_count)    
  if (len(Right_Torso_Position) == 0):
    torso_position_failure_test(bbox, engarde_length, x_min_torsoR, x_max_torsoR, y_min_torsoR, y_max_torsoR, y_average_right, 'Right', frame_count)


  # Averages the Left and Right x,y positions for engarde
  # Left Bell Guard engarde position
  x = 0
  y = 0
  if len(Left_Position) > 0:
    for i in range(len(Left_Position)):
      x = x + Left_Position[i][0]
      y = y + Left_Position[i][1]
    x = int(x/(len(Left_Position)))
    y = int(y/(len(Left_Position)))
    Left_Position = [x,y]

  if verbose == True:
    display(f'Left_Position at Engarde is:')
    display(Left_Position)

  # Right Bell Guard engarde position
  x = 0
  y = 0
  if len(Right_Position) > 0:
    for i in range(len(Right_Position)):
      x = x + Right_Position[i][0]
      y = y + Right_Position[i][1]
    x = int(x/(len(Right_Position)))
    y = int(y/(len(Right_Position)))
    Right_Position = [x,y]

  if verbose == True:
    display(f'Right_Position at Engarde is:')
    display(Right_Position)

  # Scoring_Box engarde position
  x = 0
  y = 0
  if len(Scoring_Box_Position) > 0:
    for i in range(len(Scoring_Box_Position)):
      x = x + Scoring_Box_Position[i][0]
      y = y + Scoring_Box_Position[i][1]
    x = int(x/(len(Scoring_Box_Position)))
    y = int(y/(len(Scoring_Box_Position)))
    Scoring_Box_Position = [x,y]

  if Scoring_Box_Position == [0,0]:
    Tracking_Bounding_Boxes_Temp[2] = [0,0,0,0]

  if verbose == True:
    display(f'Scoring_Box_Position at Engarde is:')
    display(Scoring_Box_Position)

  # Left_Torso engarde position
  x = 0
  y = 0
  if len(Left_Torso_Position) > 0:
    for i in range(len(Left_Torso_Position)):
      x = x + Left_Torso_Position[i][0]
      y = y + Left_Torso_Position[i][1]
    x = int(x/(len(Left_Torso_Position)))
    y = int(y/(len(Left_Torso_Position)))
    Left_Torso_Position = [x,y]

  if verbose == True:
    display(f'Left_Torso_Position at Engarde is:')
    display(Left_Torso_Position)

  # Right_Torso engarde position
  x = 0
  y = 0
  if len(Right_Torso_Position) > 0:
    for i in range(len(Right_Torso_Position)):
      x = x + Right_Torso_Position[i][0]
      y = y + Right_Torso_Position[i][1]
    x = int(x/(len(Right_Torso_Position)))
    y = int(y/(len(Right_Torso_Position)))
    Right_Torso_Position = [x,y]

  # Left_Foot engarde position
  x = 0
  y = 0
  if len(Left_Foot_Position) > 0:
    for i in range(len(Left_Foot_Position)):
      x = x + Left_Foot_Position[i][0]
      y = y + Left_Foot_Position[i][1]
    x = int(x/(len(Left_Foot_Position)))
    y = int(y/(len(Left_Foot_Position)))
    Left_Foot_Position = [x,y]

  # Right_Foot engarde position
  x = 0
  y = 0
  if len(Right_Foot_Position) > 0:
    for i in range(len(Right_Foot_Position)):
      x = x + Right_Foot_Position[i][0]
      y = y + Right_Foot_Position[i][1]
    x = int(x/(len(Right_Foot_Position)))
    y = int(y/(len(Right_Foot_Position)))
    Right_Foot_Position = [x,y]

  if verbose == True:
    display(f'Right_Torso_Position at Engarde is:')
    display(Right_Torso_Position)

  return (Left_Position, Right_Position, Scoring_Box_Position, scoring_box_size_average, Tracking_Bounding_Boxes, Left_Torso_Position, Right_Torso_Position, left_torso_size_average, right_torso_size_average, All_Bell_Guard_Positions, Left_Foot_Position, Right_Foot_Position, Foot_Size)

In [ ]:
#@title Draw_Bell_Guard_Position
def draw_Bell_Guard_Position(Left_Position, Right_Position, Scoring_Box_Position, scoring_box_size_average, Left_Torso_Position, Right_Torso_Position, Left_Foot_Position, Right_Foot_Position, frame_count, Tracking_Bounding_Boxes, video_filename, capture_width, capture_height, engarde_length, score_box_empty, camera_steady, camera_motion_threshold, Exclusion_Areas, simple_clip_vector, save_image_list):
  #Adds an overlay on the image to visualize the location of tracked objects

  path = r'/content/Mask_RCNN/videos/'
  capture = cv2.VideoCapture(os.path.join(path, video_filename))

  capture.set(cv2.CAP_PROP_FRAME_WIDTH, capture_width)
  capture.set(cv2.CAP_PROP_FRAME_HEIGHT, capture_height)

  #Color format is [B,G,R]
  left_light_color_default = [[],[],[]]
  right_light_color_default = [[],[],[]]
  left_light_color = []
  right_light_color = []

  # Creates a list of Files from a Directory
  path = r'/content/Mask_RCNN/videos/save/'


  left_light_comparison, right_light_comparison, default_color = [], [], []

  # for i, file in enumerate(files):
  for i in range(len(save_image_list)):
    # Reads the image
    # name = os.path.join(path, file)
    img = save_image_list[i]

    # OpenCV uses Blue, Green, Red order
    # Light_Color is of the format [[[B0],[G0],[R0]],[[B1],[G1],[R1]],[[B2],[G2],[R2]],...]
    
    if i <= engarde_length:
      if scoring_box_size_average == [0,0]:
        scoring_box_size_average = [int(capture_width/5), int(capture_height/5)]
      if verbose == True:
        display(f'The average scoring box size is {scoring_box_size_average}.')
      # Uses a comparison of frames and scoring box position to determine the light off colors

      if verbose == True:
        display(f'The index i is {i}.')
        display(f'The Score Box Position is: {Scoring_Box_Position}.')
        display(f'The Score Box Position at i is {Scoring_Box_Position[i]}.')
      [left_light_comparison_temp, right_light_comparison_temp, defualt_color_temp] = scoring_box_lights(img, Scoring_Box_Position[i], scoring_box_size_average, [], i, score_box_empty)
      left_light_comparison.append(left_light_comparison_temp)
      right_light_comparison.append(right_light_comparison_temp)
      default_color.append(defualt_color_temp)
      # Averages the Default Color on the Last iteration
      if i == engarde_length:
        b_temp = int(sum(default_color[0])/len(default_color[0]))
        g_temp = int(sum(default_color[1])/len(default_color[1]))
        r_temp = int(sum(default_color[2])/len(default_color[2]))
        default_color = [b_temp,g_temp,r_temp]
    elif i > engarde_length:
      try:
        [left_light_comparison_temp, right_light_comparison_temp, defualt_color_temp] = scoring_box_lights(img, Scoring_Box_Position[i], scoring_box_size_average, default_color, i, score_box_empty)
      except:
        if verbose == True:
          display(f'Light Comparison Failed due to Error at frame {i}.')
        [left_light_comparison_temp, right_light_comparison_temp, defualt_color_temp] = [0,0,[]]
      left_light_comparison.append(left_light_comparison_temp)
      right_light_comparison.append(right_light_comparison_temp)

    if verbose == True:
      display(f'The Left Position is: {Left_Position}.')
      display(f'The iterator i is {i}.')

    #Creates the dots on the Bell Guards
    frame = cv2.circle(img, (Left_Position[i][0], Left_Position[i][1]), int(4*(capture_width/1280)), (118, 37, 217), -1)
    frame = cv2.circle(frame, (Right_Position[i][0], Right_Position[i][1]), int(4*(capture_width/1280)), (157, 212, 19), -1)
    frame = cv2.circle(frame, (Scoring_Box_Position[i][0], Scoring_Box_Position[i][1]), int(4*(capture_width/1280)), (255, 255, 0), -1)
    frame = cv2.circle(frame, (Left_Torso_Position[i][0], Left_Torso_Position[i][1]), int(4*(capture_width/1280)), (0, 255, 0), -1)
    frame = cv2.circle(frame, (Right_Torso_Position[i][0], Right_Torso_Position[i][1]), int(4*(capture_width/1280)), (255, 255, 0), -1)
    # frame = cv2.circle(frame, (Left_Foot_Position[i][0], Left_Foot_Position[i][1]), int(4*(capture_width/1280)), (118, 37, 217), -1)
    # frame = cv2.circle(frame, (Right_Foot_Position[i][0], Right_Foot_Position[i][1]), int(4*(capture_width/1280)), (157, 212, 19), -1)
    
    # Creates the Representative Bell Guard Position
    frame = cv2.circle(img, (Left_Position[i][0], int(capture_height/2)), int(20*(capture_width/1280)), (118, 37, 217), -1)
    frame = cv2.circle(frame, (Right_Position[i][0], int(capture_height/2)), int(20*(capture_width/1280)), (157, 212, 19), -1)
    # frame = cv2.circle(frame, (Left_Foot_Position[i][0], int(capture_height*5/8)), int(10*(capture_width/1280)), (118, 37, 217), -1)
    # frame = cv2.circle(frame, (Right_Foot_Position[i][0], int(capture_height*5/8)), int(10*(capture_width/1280)), (157, 212, 19), -1)

    # Creates the Light Indicators
    rect_size = int(capture_width/40)
    if (simple_clip_vector[i][2] == 1):
      #Creates the Left Score Light
      frame = cv2.rectangle(frame, (rect_size, int(rect_size*1.5)), (rect_size*5, int(rect_size*4.5)), (0, 0, 255), -1)
    if (simple_clip_vector[i][3] == 1):
      #Creates the Right Score Light
      frame = cv2.rectangle(frame, (capture_width - rect_size, int(rect_size*1.5)), (capture_width - rect_size*5, int(rect_size*4.5)), (0, 255, 0), -1)

    # Adds Frame Number to the Image
    text = 'Frame' + str(i)
    frame = cv2.putText(frame, text, (int(capture_width*7/8), int(capture_height*1/16)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, )
    # Adds if the Camera Motion is detected by difference images
    if camera_steady[i] > camera_motion_threshold:
      text = 'Camera'
      frame = cv2.putText(frame, text, (int(capture_width*7/8), int(capture_height*3/16)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 4, )
      text = 'Motion'
      frame = cv2.putText(frame, text, (int(capture_width*7/8), int(capture_height*4/16)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 4, )
    
    # Sets BellGuard Position Colors
    left_color = (255, 0, 0)
    right_color = (0, 255, 0)

    #Creates the Tracking Boxes
    frame = cv2.putText(frame, 'Tracking Box', (Tracking_Bounding_Boxes[i][0][0], Tracking_Bounding_Boxes[i][0][2]), cv2.FONT_HERSHEY_COMPLEX, 0.7, left_color, 2)
    frame = cv2.rectangle(frame, (Tracking_Bounding_Boxes[i][0][0], Tracking_Bounding_Boxes[i][0][2]),(Tracking_Bounding_Boxes[i][0][1], Tracking_Bounding_Boxes[i][0][3]),left_color, 2)
    frame = cv2.putText(frame, 'Tracking Box', (Tracking_Bounding_Boxes[i][1][0], Tracking_Bounding_Boxes[i][1][2]), cv2.FONT_HERSHEY_COMPLEX, 0.7, right_color, 2)
    frame = cv2.rectangle(frame, (Tracking_Bounding_Boxes[i][1][0], Tracking_Bounding_Boxes[i][1][2]),(Tracking_Bounding_Boxes[i][1][1], Tracking_Bounding_Boxes[i][1][3]),right_color, 2)

    # [frame, none] = overlay_keypoints(frame, keypoints[i][0], keypoints[i][1], True)

    #Draws the Exclusion Areas
    for j in range(len(Exclusion_Areas)):
      frame = cv2.circle(frame, (Exclusion_Areas[j][0],Exclusion_Areas[j][1]), int(capture_width/80), (144,238,144), 2)

    if verbose == True:
      display(f'The Tracking Box for the Left Fencer at frame {i} is:')
      display(f'{Tracking_Bounding_Boxes[i][0][0]},{Tracking_Bounding_Boxes[i][0][2]}')
      display(f'The Tracking Box for the Right Fencer at frame {i} is:')
      display(f'{Tracking_Bounding_Boxes[i][1][0]},{Tracking_Bounding_Boxes[i][1][2]}')

    #Saves the image frame overwriting the original image
    # name = os.path.join(path, file)
    file = str(i) + '.jpg'
    name = os.path.join(path, file)
    # cv2.imwrite(name, frame)
    save_image_list[i] = frame

    if verbose == True:
      display(f'The Draw Bell Guard frame {i} is being saved at {name}.')

  #Releases capture so that other files can be used
  capture.release()

  return (left_light_comparison, right_light_comparison, save_image_list)

In [ ]:
def create_title_card(width, height, results, pred_total, name, img_directory):

  # Tests for and creates if needed, a folder for the font
  os.chdir('/content/')
  if not exists('/content/font'):
    os.mkdir('font')
  os.chdir('/content/font')
  if not exists('NotoSerif-hinted.zip'):
    !wget https://noto-website-2.storage.googleapis.com/pkgs/NotoSerif-hinted.zip
    !unzip "NotoSerif-hinted.zip"
  os.chdir('/content/')

  # create same size image of background color
  bg_color = (0,0,0)
  # bg = np.full((img.shape), bg_color, dtype=np.uint8)
  image = np.full((height, width,3), bg_color, dtype=np.uint8)

  # draw text on bg
  text_color = (255,255,255)
  font = cv2.FONT_HERSHEY_TRIPLEX

  scaling = width/1280
  fontScale_header= 3.8*scaling
  fontScale = 3.6*scaling
  thickness = int(4*scaling)
  certainty_cutoff = 0.1
  

  from PIL import Image

  if (pred_total * 100) > certainty_cutoff:
    # Creates the Text for the Title Card
    text1 = "Predicted Result"
    text2 = 'Left: ' + str(results[0]) + '%'
    text3 = 'Right: ' + str(results[1]) + '%'
    text4 = 'Simul: ' + str(results[2]) + '%'

    # Gets the boundary of the text. It is approximate since it uses Hershey Text
    text1size = cv2.getTextSize(text1, font, fontScale_header, thickness)[0]
    text2size = cv2.getTextSize(text2, font, fontScale, thickness)[0]
    text3size = cv2.getTextSize(text3, font, fontScale, thickness)[0]
    text4size = cv2.getTextSize(text4, font, fontScale, thickness)[0]

    # Defines the x and y values of the Text Blocks
    text1x = int((image.shape[1] - text1size[0]) / 2)
    text1y = int(height*0/10)
    text2x = int((image.shape[1] - text2size[0]) / 2)
    text2y = int(height*2/10)
    text3x = int((image.shape[1] - text3size[0]) / 2)
    text3y = int(height*4/10)
    text4x = int((image.shape[1] - text4size[0]) / 2)
    text4y = int(height*6/10)

    # Uses PIL for a so that a custom font can be used
    im = Image.fromarray(np.uint8(image))
    font_pil_header = PIL.ImageFont.truetype('/content/font/NotoSerif-SemiBold.ttf', int(scaling*140))
    font_pil = PIL.ImageFont.truetype('/content/font/NotoSerif-SemiBold.ttf', int(scaling*130))
    draw  = PIL.ImageDraw.Draw(im)

    # Draws the Text
    draw.text((text1x, text1y), text1, fill=(255,255,255,255), font=font_pil_header)
    draw.text((text2x, text2y), text2, fill=(255,255,255,255), font=font_pil)
    draw.text((text3x, text3y), text3, fill=(255,255,255,255), font=font_pil)
    draw.text((text4x, text4y), text4, fill=(255,255,255,255), font=font_pil)

  else:
    # Creates the Text for an Uncertain Result
    text1 = "Certainty is"
    text2 =  "too Low"
    # Gets the boundary of the text. It is approximate since it uses Hershey Text
    text1size = cv2.getTextSize(text1, font, fontScale, thickness)[0]
    text2size = cv2.getTextSize(text2, font, fontScale, thickness)[0]
    text1x = int((image.shape[1] - text1size[0]) / 2)
    text1y = int(height*0/10)
    text2x = int((image.shape[1] - text2size[0]) / 2)
    text2y = int(height*2/10)

    im = Image.fromarray(np.uint8(image))
    font_pil_header = PIL.ImageFont.truetype('/content/font/NotoSerif-SemiBold.ttf', int(scaling*140))
    font_pil = PIL.ImageFont.truetype('/content/font/NotoSerif-SemiBold.ttf', int(scaling*130))
    draw  = PIL.ImageDraw.Draw(im)
    draw.text((text1x, text1y), text1, fill=(255,255,255,255), font=font_pil)
    draw.text((text2x, text2y), text2, fill=(255,255,255,255), font=font_pil)

  # Converts the PIL image back into Numpy
  image = np.array(im)

  # Finds the Number of the Last frame
  ROOT_DIR = '/content/Mask_RCNN/'
  VIDEO_DIR = os.path.join(ROOT_DIR, "videos")
  VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, img_directory)
  images = list(glob.iglob(os.path.join(VIDEO_SAVE_DIR, '*.*')))
  # Sorts the images by integer index
  images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))

  last_iterator = int(images[-1].split(img_directory + '/')[1][:-4])
  display(f'The last image is {last_iterator}.')

  # Saves the image
  for i in range(25):
    save_name = name + str(i + last_iterator + 1) + '.jpg'
    save_name = os.path.join(VIDEO_SAVE_DIR, save_name)
    cv2.imwrite(save_name, image)

  return

In [ ]:
#@title Mask_Image
def mask_image(frame, width, height, masking_box):
  # Used to Mask parts of the image that are not of interest

  if verbose == True:
    display(f'The masking box is:')
    display(masking_box)

  #Create the Mask
  mask = np.zeros((height, width, 3), dtype = np.uint8);
  for i in range(len(masking_box)):
    mask = cv2.rectangle(mask, (masking_box[i][0], masking_box[i][2]) ,(masking_box[i][1], masking_box[i][3]), (255,255,255), -1)

  #Applies the mask to Frame
  frame = cv2.bitwise_and(mask, frame)

  return (frame)

In [ ]:
#@title Create_Representative_Image
def create_representative_image(clip_vector, capture_width, capture_height):
  # Allows for an overlay that represents the bellguard horizontal motion and box lights

  #Creates a Folder to save the images and removes previous version
  os.chdir('/content/Mask_RCNN/videos')
  # Removes and Recreates the Save_White_Dot to ensure the directory is empty
  try:
    shutil.rmtree('save_white_dot')
    if verbose == True:
      display(f'Removed the Save_White_Dot folder.')
  except:
    if verbose == True:
      display(f'ERROR removing the Save_White_Dot folder.')
  os.mkdir('save_white_dot')

  rect_size = int(capture_width/40)

  #Defines the File Path
  path = r'/content/Mask_RCNN/videos/save_white_dot/'
  
  for i in range(len(clip_vector)):
    img = np.zeros((capture_height,capture_width,3), np.uint8)

    #Creates the Left Bell_Guard
    img = cv2.circle(img, (clip_vector[i][0], int(capture_height/2)), 20, (118, 37, 217), -1)
    #Creates the Right Bell_Guard
    img = cv2.circle(img, (clip_vector[i][1], int(capture_height/2)), 20, (157, 212, 19), -1)

    if (clip_vector[i][2] == 1):
      #Creates the Left Score Light
      img = cv2.rectangle(img, (rect_size, rect_size), (rect_size*5, rect_size*3), (0, 0, 255), -1)
    if (clip_vector[i][3] == 1):
      #Creates the Right Score Light
      img = cv2.rectangle(img, (capture_width - rect_size, rect_size), (capture_width - rect_size*5, rect_size*3), (0, 255, 0), -1)

    name = str(i) + '.jpg'
    name = os.path.join(path, name)

    cv2.imwrite(name, img)

  return

In [ ]:
#@title Create_Overlay_Image
def create_overlay_image(frame_count):
  # Allows for an overlay that represents the bellguard horizontal motion and box lights

  #Creates a Folder to save the images and removes previous version
  os.chdir('/content/Mask_RCNN/videos/')
  # !rm -r /content/Mask_RCNN/videos/overlay
  # Attempts to remove the Overlay folder and recreate it to ensure that it is empty
  try:
    shutil.rmtree('overlay')
  except:
    display(f'ERROR removing the Overlay folder.')
  # !mkdir overlay
  ch


  #Defines the File Path
  path = r'/content/Mask_RCNN/videos/overlay/'
  path_background = r'/content/Mask_RCNN/videos/save/'
  path_foreground = r'/content/Mask_RCNN/videos/save_white_dot/'
  for i in range(frame_count):
    background_name = str(i) + '.jpg'
    background_name = os.path.join(path_background, background_name)

    foreground_name = str(i) + '.jpg'
    foreground_name = os.path.join(path_foreground, foreground_name)
    
    background = cv2.imread(background_name)
    foreground = cv2.imread(foreground_name)

    added_image = cv2.addWeighted(background,0.8,foreground,1.0,0)

    combined_name = str(i) + '.jpg'
    combined_name = os.path.join(path, combined_name)

    if verbose == True:
      display(f'The file added image is saved at {combined_name}.')

    cv2.imwrite(combined_name, added_image)

  return

In [ ]:
#@title Light_Color_Comparison
def light_color_comparison(light_color, light_color_default, color):
  # Deterines if a light turned on based on a default color, an input color and expected color

  light_comparison = []
  # A high max distance is less sensitive and a lower max distance is more sensitive
  max_distance_total = 180
  max_distance_specific_color = 90

  if color == 'Red':
    color_specific = 2
  elif color == 'Green':
    color_specific = 1
  else:
    pass

  if verbose == True:
    display(f'The Color being analyzed is {color}.')
    display(f'The default color is:')
    display(light_color_default)
    display(f'With the specific color being {light_color_default[color_specific]}')
    display(f'The max distance total is {max_distance_total}.')
    display(f'The max distance for a specific color is {max_distance_specific_color}.')

  #i cycles through each light value corresponding to each frame
  for i in range(len(light_color)):
    distance = 0
    for j in range(3):
      distance = distance + (light_color[i][j] - light_color_default[j])**2

    distance_specific_color = abs(light_color[i][color_specific] - light_color_default[color_specific])

    distance = int((distance)**(0.5))
    if vebose == True:
      display(f'The distance is {distance} and the color specific distance is {distance_specific_color} for frame {i}.')
    #0 is no color change from the default color)
    if (distance > max_distance_total and distance_specific_color > max_distance_specific_color):
      light_comparison.append(1)
      if verbose == True:
        display(f'The light is ON.')
    #1 is a color change from the default color
    else:
      light_comparison.append(0)
      if verbose == True:
        display(f'The light is OFF.')

  return (light_comparison)

In [ ]:
#@title Clip_Vector_Generator
def clip_vector_generator(Left_Position, Right_Position, left_light_comparison, right_light_comparison, clip_vector_previous, width):
  #Compiles the clip_vector that is used for the action analysis

  # Allows for the assumption that both lights are on if the positions are close to each other.
  # Useful if there is difficulty detecting the scoring box.
  close_bellguards = False
  # Once lights turn on it is assumed the lights stay on for the rest of the action
  light_assumption = False

  if len(Left_Position) != len(Right_Position):
    display(f'The Left and Right Positions do not match up')
  else:
    pass

  # This is either [] or the Previously saved Clip_Vector
  clip_vector = clip_vector_previous

  for i in range(len(Left_Position)):  
    # Checks if the lights should be assumed on if they are not already
    # Determines if the bellguards are close to each other
    # if (abs(Left_Position[i][0] - Right_Position[i][0]) < width*.050) and (light_assumption == False):
    if ((Right_Position[i][0] - Left_Position[i][0]) < width*position_difference_ratio) and (light_assumption == False):
      close_bellguards = True

    # Adjusts the clip vector to reflect scoring box light assumptions
    clip_vector_temp = [[],[],[],[]]
    clip_vector_temp[0] = Left_Position[i][0]
    clip_vector_temp[1] = Right_Position[i][0]
    if (assume_lights == True and close_bellguards == True) or light_assumption == True:
      clip_vector_temp[2] = 1
      clip_vector_temp[3] = 1
      light_assumption = True
    else:
      if ignore_box_lights == True:
        clip_vector_temp[2] = 0
        clip_vector_temp[3] = 0
      else:
        clip_vector_temp[2] = left_light_comparison[i]
        clip_vector_temp[3] = right_light_comparison[i]

    clip_vector.append(clip_vector_temp)

  return (clip_vector)

In [ ]:
def simple_clip_vector_generator(Left_Position, Right_Position, width):
  #Compiles the clip_vector that is used for the action analysis

  # Allows for the assumption that both lights are on if the positions are close to each other.
  # Useful if there is difficulty detecting the scoring box.
  close_bellguards = False
  # Once lights turn on it is assumed the lights stay on for the rest of the action
  light_assumption = False

  if verbose == True:
    display(f'The Left Position is {Left_Position}.')
    display(f'The Right Position is {Right_Position}.')

  if len(Left_Position) != len(Right_Position):
    display(f'The Left and Right Positions do not match up')
  else:
    pass

  clip_vector = []

  for i in range(len(Left_Position)):  
    # Checks the lights should be assumed on if they are not already
    # Determines if the bellguards are close to each other
    if ((Right_Position[i][0] - Left_Position[i][0]) < width*position_difference_ratio) and (light_assumption == False):
      close_bellguards = True

    # Adjusts the clip vector to reflect scoring box light assumptions
    clip_vector_temp = [[],[],[],[]]
    clip_vector_temp[0] = Left_Position[i][0]
    clip_vector_temp[1] = Right_Position[i][0]
    if (assume_lights == True and close_bellguards == True) or light_assumption == True:
      clip_vector_temp[2] = 1
      clip_vector_temp[3] = 1
      light_assumption = True
    else:
      clip_vector_temp[2] = 0
      clip_vector_temp[3] = 0

    clip_vector.append(clip_vector_temp)

  return (clip_vector)

In [ ]:
#@title Clip_Vector_Np_Save
def clip_vector_np_save(clip_call, file_number, clip_vector, file_title):
  # Saves the clip vector for future use
  # Clip_Call Left_Touch, Right_Touch, Simul

  # Generates the clip_vector speed based on the clip_vector
  clip_vector_speed = []
  for i in range(len(clip_vector)-1):
    clip_vector_speed.append([])
    clip_vector_speed[i].append(clip_vector[i+1][0]-clip_vector[i][0])
    # Reverses the Right Fencers position so that positive is towards the opponent
    clip_vector_speed[i].append(clip_vector[i][1]-clip_vector[i+1][1])
    clip_vector_speed[i].append(clip_vector[i+1][2])
    clip_vector_speed[i].append(clip_vector[i+1][3])

  # Generates the clip_vector acceleration based on the clip_vector
  clip_vector_acceleration = []
  for i in range(len(clip_vector_speed)-1):
    clip_vector_acceleration.append([])
    clip_vector_acceleration[i].append(clip_vector[i+1][0]-clip_vector[i][0])
    # Reverses the Right Fencers position so that positive is towards the opponent
    clip_vector_acceleration[i].append(clip_vector[i][1]-clip_vector[i+1][1])
    clip_vector_acceleration[i].append(clip_vector[i+1][2])
    clip_vector_acceleration[i].append(clip_vector[i+1][3])

  path = '/content/drive/My Drive/projects/fencing/Fencing Clips/'

  # # Saves the clip_vector_acceleration
  clip_vector_acceleration_np = np.asarray(clip_vector_acceleration)

  # Saves a Copy to the Sync Folder
  # if run_sync_folder == True or use_sync_folder == True:
  if run_entire_sync_folder == True or run_most_recent_clip == True:
    file_title = file_title[:-4]
    clip_vector_acceleration_np_name = file_title + '_acc.csv'
    sync_folder_acc = '/content/drive/My Drive/Sync/Acceleration'
    os.chdir(sync_folder_acc)
    np.savetxt(clip_vector_acceleration_np_name, clip_vector_acceleration_np, delimiter=',')
    # os.chdir(name)

  return

In [ ]:
#@title Left_Right_Test
def Left_Right_Test(Left_Position, Right_Position):
  # Requires that the Left and Right BellGuards be on the Left and Right sides respectively

  #Left_Position is chosen arbitrarily for length
  for i in range(len(Left_Position)):
    if Left_Position[i][0] > Right_Position[i][0]:
      if verbose == True:
        display(f'The Left and Right were swapped on frame {i} and are now corrected.')
      position_temp = Left_Position[i]
      Left_Position[i] = Right_Position[i]
      Right_Position[i] = position_temp
    else:
      pass

  return (Left_Position, Right_Position)

In [ ]:
#@title Camera_Motion_Adjustment
def camera_motion_adjustment(Position, Score_Box_Position):
  # Takes a Position as an input and adjusts the position to compensate for camera motion
  # Used solely the x position of the scoring box to calculate motion
  # Ignores the change in angle as the camera is rotated
  # This is only used when it is assumed that the Scoring Box is well detected and tracked

  Score_Box_Position_Temp = []
  #Converts Scoring Box Positions to solely x value
  #Scoring Box Position is of the format [x0,x1,x2...]
  for i in range(len(Score_Box_Position)):
    Score_Box_Position_Temp.append(Score_Box_Position[i][0])

  for j in range(len(Position)):
    score_box_delta = Score_Box_Position_Temp[j] - Score_Box_Position_Temp[0]
    Position[j][0] = Position[j][0] - score_box_delta

  return (Position)

In [ ]:
#@title Position_Down_Scale
def position_down_scale(Position1, Position2, capture_width, capture_height):
  # Scales the Position Down to the Capture Width in the x axis if required for visualization convenience
  # Does not alter the Clip Vector Data
  
  position_temp = []

  for i in range(len(Position1)):
    position_temp.append(Position1[i][0])

  for j in range(len(Position2)):
    position_temp.append(Position2[j][0])

  min_x_position = min(position_temp)
  max_x_position = max(position_temp)

  if min_x_position < 0:
    #Shifts the bellguards to the right for the camera moving to the left
    for i in range(len(Position1)):
      Position1[i][0] = int(Position1[i][0] - min_x_position)

    for j in range(len(Position2)):
      Position2[j][0] = int(Position2[j][0] - min_x_position)

  # Absolute Pixel
  if max_x_position > capture_width:
    #Scales the max x position if greater than the screen
    for i in range(len(Position1)):
      Position1[i][0] = int(Position1[i][0] * capture_width / max_x_position)

    for j in range(len(Position2)):
      Position2[j][0] = int(Position2[j][0] * capture_width / max_x_position)

  return (Position1, Position2)

In [ ]:
#@title Random_Colors
def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors

In [ ]:
#@title Apply_Mask
def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image

In [ ]:
#@title Display_Instances
def display_instances(image, boxes, masks, ids, names, scores, file_name):
    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]
    colors = random_colors(n_instances)

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        pass

    for i, color in enumerate(colors):
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        # label = boxes[i][4]
        label = names[ids[i]]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]
        # display(f'The mask is: {mask}')
        # image = apply_mask(image, mask, color)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(
            image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
        )

    return image

In [ ]:
#@title Smooth_Clip_Vector
def smooth_clip_vector(clip_vector, engarde_length):
  # Allows for smoothing the clip_vector

  a = []
  b = []
  for i in range(engarde_length, len(clip_vector)):
    a.append(clip_vector[i][0])
    b.append(clip_vector[i][1])

  x = np.linspace(engarde_length,len(clip_vector), len(clip_vector) - engarde_length)

  # sos = signal.ellip(13, 0.009, 80, 0.05, output='sos')
  # yhata = signal.sosfilt(sos, a)
  if len(a)%2 == 1:
    yhata = signal.savgol_filter(a, len(a), 11)
    yhatb = signal.savgol_filter(b, len(b), 11)
  else:
    yhata = signal.savgol_filter(a, len(a) - 1, 11)
    yhatb = signal.savgol_filter(b, len(b) - 1, 11)    

  # plt.plot(x,a, color='black')
  # plt.plot(x,yhata, color='red')
  plt.plot(x,b, color='black')
  plt.plot(x,yhatb, color='blue')
  plt.show()

  vector_clip_smooth = []

  for j in range(len(clip_vector)):
    if j <= engarde_length:
      clip_vector_smooth_temp = [clip_vector[j][0], clip_vector[j][1], clip_vector[j][2], clip_vector[j][3]]
    else:
      clip_vector_smooth_temp = [int(yhata[j - engarde_length]), int(yhatb[j - engarde_length]), clip_vector[j][2], clip_vector[j][3]]
    vector_clip_smooth.append(clip_vector_smooth_temp)

  return (vector_clip_smooth)

In [ ]:
#@title Load_Clip_Vector
def load_clip_vector():
  # Only used for large clips

  display(f'Loading the Clip Vector...')
  filename = r'/content/drive/My Drive/projects/fencing/Fencing Clips/Temp_Clip_Vector/Temp_Clip_Vector_Clips/clip_vector_np1.csv'

  display(f'Attempting to load:')
  display(filename)
  try:
    vector_data = pd.read_csv(filename, header=None)
    arr = vector_data.to_numpy(dtype = np.int32)
    clip_vector = arr.tolist()
  except:
    display(f'Load Failure...')
    display(f'The clip_vector did not exist so it is set to []')
    clip_vector = []

  return (clip_vector)

In [ ]:
#@title Create_Tracking_Masks
def create_tracking_masks(previous_positions, certainty, frame_count, torso_size, width, height):
  #Creates Tracking Boxes that can be used to mask the image, ignoring parts that are not of interest
  #Format, Tracking_Boxes = [Left, Right, Scorebox], Left = [x_min, x_max, y_min, y_max]
  #Format, Previous Positions
          # previous_positions  = [[Left_Position[-1], Left_Position[-2]], \
          #                      [Right_Position[-1], Right_Position[-2]], \
          #                      [Scoring_Box_Position[-1], Scoring_Box_Position[-2]], \
          #                      [Left_Torso_Position[-1], Left_Torso_Position[-2]], \
          #                      [Right_Torso_Position[-1], Right_Torso_Position[-2]]]

          #Format, positions are [x,y]

  #Format, torso_position = [[Left_x,Lefty],[Right_x,Right_y]]
  #Format, torso_size = [[Lw,Lh], [Rw,Rh]]

  if verbose == True:
    display(f'Creating Tracking Masks...')
    display(f'The Previous Positions are:')
    display(previous_positions)
    display(f'The torso sizes are:')
    display(torso_size)

  frame_mask = []

  #Certainty is the number of times the bellguard has not been detected in previous frames
  #certainty_default is the minimum size of the tracking box
  certainty_default = int(width/16)
  #certainty_multiplier is how much the tracking box enlarges following a missed
  certainty_multiplier = int(width/80)
  y_limiter = 24

  #Max allowed speed of a bellguard in a single frame
  max_speed = int(width/48)

  if verbose == True:
    display(f'The length of the previous positions is: {len(previous_positions)}.')

  for i in range(len(previous_positions)):
    if verbose == True:
      display(f'The masking iteration for frame {frame_count} is {i}.')
    #FINDS THE LEFT MASKING BOX
    x_pos = previous_positions[i][0][0]
    y_pos = previous_positions[i][0][1]
    #Converts previous position into a speed
    x_speed = min(previous_positions[i][0][0] - previous_positions[i][1][0], max_speed)
    # Limits the maximum vertical speed with relation to x
    y_speed = min(previous_positions[i][0][1] - previous_positions[i][1][1], int(max_speed/y_limiter))

    if verbose == True:
      display(f'x and y position is ({x_pos},{y_pos}) and the speeds are ({x_speed},{y_speed}).')

    x_min = x_pos + (x_speed) - (certainty[i]*certainty_multiplier) - certainty_default
    x_max = x_pos + (x_speed) + (certainty[i]*certainty_multiplier) + certainty_default
    y_min = y_pos + (y_speed) - (certainty[i]*certainty_multiplier) - certainty_default
    y_max = y_pos + (y_speed) + (certainty[i]*certainty_multiplier) + certainty_default

    #Appends the mask to collection of tracked areas
    frame_mask.append([x_min, x_max, y_min, y_max])

  if verbose == True:
    display(f'The Frame Mask for frame {frame_count} is:')
    display(frame_mask)

  return(frame_mask)

In [ ]:
def make_video(outvid, images=None, fps=25, size=None,
               is_color=True, format="FMP4"):
  
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
 
    The function relies on http://opencv-python-tutroals.readthedocs.org/en/latest/.
    By default, the video will have the size of the first image.
    It will resize every image to this size before adding them to the video.
    """
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid

In [ ]:
def mean_of_a_numpy_percentile(arr, percentile_cutoff):
  # Returns a percentile value of a numpy array

  if verbose == True:
    display(f'The average of arr is {np.average(arr)}.')

  percentile_value = np.percentile(arr, percentile_cutoff)

  # Uses just the percentile without averaging
  array_percentile_mean = percentile_value

  return (array_percentile_mean)

In [ ]:
#@title Frame_Comparison_SSIM
def frame_comparison_ssim(frame_number, frame, width, height, engarde_length):

  # Determines if the subsequent frame is identical to the current or if there was camera motion
  # Uses Mean Square Error and Structural Similarity Index

  save_path = r'/content/Mask_RCNN/videos/original/'
  # image_num = frame_number
  image_name1 = str(frame_number-1) + '.jpg'
  file_name1 = os.path.join(save_path, image_name1)
  # file_name2 = os.path.join(save_path, image_name2)

  image1 = cv2.imread(file_name1)
  image2 = frame

  # Uses a tighter crop for engarde positioning to minimize motion outside the bout
  if frame_number <= engarde_length:
    crop_image1 = image1[int(height*1/5):int(height*3/4), 0:width]
    crop_image2 = image2[int(height*1/5):int(height*3/4), 0:width]
  else:
    # Removes the bottom of the frame to minimize the effect of overlays and shadowing in the foreground
    crop_image1 = image1[int(height*0):int(height*2/4), 0:width]
    crop_image2 = image2[int(height*0):int(height*2/4), 0:width]

  # Calculate MSE
  m = np.linalg.norm(image1 - image2)
  
  # # If GrayScale
  # s = ssim(imageA, imageB)
  # If Color
  s = ssim(crop_image1, crop_image2, multichannel=True)

  if verbose == True:
    display(f'The Mean Square Error of frame {frame_number} is {m}.')
    display(f'The Structural Similarity Index of frame {frame_number} is {s}.')

  return(m, s)

In [ ]:

#@title Frame_Comparison
def frame_comparison(frame_number, frame, width, height, engarde_length, original_image_list):
  # Determines if the subsequent frame is identical to the current or if there was camera motion
  # By calculating an average Hue from an HSV image. The Hue is then correlated to an average 
  # color difference between frames.

  image1 = original_image_list[frame_number - 1]
  image2 = frame

  # Uses a tighter crop for engarde positioning to minimize motion outside the bout
  if frame_number <= engarde_length:
    crop_image1 = image1[int(height*1/5):int(height*3/4), 0:width]
    crop_image2 = image2[int(height*1/5):int(height*3/4), 0:width]
  else:
    # Removes the bottom of the frame to minimize the effect of overlays and shadowing in the foreground
    crop_image1 = image1[int(height*0):int(height*3/4), 0:width]
    crop_image2 = image2[int(height*0):int(height*3/4), 0:width]

  #Convert to Grayscale and find the Difference
  image1_gray = cv2.cvtColor(crop_image1, cv2.COLOR_BGR2GRAY)
  image2_gray = cv2.cvtColor(crop_image2, cv2.COLOR_BGR2GRAY)
  

  # Finds the HSV of image2
  image2_HSV = cv2.cvtColor(image2, cv2.COLOR_BGR2HSV)
  h_average = np.average(image2_HSV[0])

  # Uses Uncropped Frames
  # image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
  # image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
  # image_diff = cv2.absdiff(image1_gray,image2_gray)
  image_diff = cv2.absdiff(crop_image1,crop_image2)
  # image_diff_color = cv2.absdiff(image1,image2)

  if verbose == True:
    display(f'The max for the difference of frame {frame_number} is {np.amax(image_diff)}.')
    display(f'The average/median for the difference of frame {frame_number} is {np.average(image_diff)}/{np.median(image_diff)}.')

  if verbose == True:
    display(f'The shape of image_diff is {image_diff.shape}.')

  average_image_diff = np.average(image_diff)
  if verbose == True:
    display(f'The image difference average is {np.average(image_diff)} for frame {frame_number}.')

  # average_diff = np.average(image_diff)
  # average_diff = np.average(image_diff_color)

  return(average_image_diff, h_average)

In [ ]:
def read_video(file_name, folder):
  # Reads a video file and returns the video as a list of arrays for each frame and width, height, fps

  display(f'The file to be read is {os.path.join(folder, file_name)}')

  video_array_list = []
  capture = cv2.VideoCapture(os.path.join(folder, file_name))
  
  fps = int(capture.get(cv2.CAP_PROP_FPS))
  # Prevents unusually large FPS
  if fps > 100:
    fps = 30

  # capture.set(cv2.CAP_PROP_FRAME_WIDTH, width)
  # capture.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

  while True:
    ret, frame = capture.read()
    if not ret:
      break

    video_array_list.append(frame)

  capture.release()

  return(video_array_list, fps)

In [ ]:
#@title Test_and_Remove_Duplicate_Frames
def test_and_remove_duplicate_frames(original_image_list, engarde_length, remove_duplicate_frames):
  # Creates a List of unique frames with by comparing the previous and current frames
  # This compensates for video compression that may give duplicate frames when FPS is changed

  camera_steady = []
  engarde_diff_average_arr = np.array([])
  engarde_hue_average_arr = np.array([])

  # original_image_list = []
  original_image_worpt_list = []

  #Tests for the Same Frames and removes identical frames
  # frame_count = 0
  num_of_deleted_frames = 0
  frame_test = True

  # while True:
  #   ret, frame = capture.read()
  #   if not ret:
  #     break
  #   if verbose == True:
  #     display(f'frame_count is {frame_count}.')

  for i in range(len(original_image_list)):
    # #Saves an original version of the frame without Regions of Interest

    if i > 0:
      if verbose == True:
        display(f'Performing Difference Check for Frame {i}')
      # True implies a unique frame while False is a repeat
      # [average_diff, h_average] = frame_comparison(frame_count, frame, width, height, frame_count, engarde_length, original_image_list)
      [average_diff, h_average] = frame_comparison(i, original_image_list[i], width, height, engarde_length, original_image_list)

      # Uses the Engarde Positioning to determine a baseline difference level between frames
      if i < engarde_length:
        # engarde_diff_average_list.append(average_diff)
        engarde_diff_average_arr = np.append(engarde_diff_average_arr, average_diff)
        engarde_hue_average_arr = np.append(engarde_hue_average_arr, h_average)

      elif i == engarde_length:

        engarde_diff_average_arr = np.append(engarde_diff_average_arr, average_diff)
        engarde_hue_average_arr = np.append(engarde_hue_average_arr, h_average)

        # Emperically Derived Threshold Based on Hue
        duplicate_threshold = np.average(engarde_hue_average_arr) * -.037+3.8663
        # duplicate_threshold = np.average(engarde_hue_average_arr) * -.037+ 1.5663
        camera_motion_threshold = np.percentile(engarde_diff_average_arr, 40) * camera_motion_threshold_factor
                
        if verbose == True:
          display(f'The Duplicate Threshold at the Engarde Length is {duplicate_threshold}.')
          display(f'The Camera Motion Threshold at the Engarde Length is {camera_motion_threshold}.')

      elif i > engarde_length:
        if average_diff < duplicate_threshold:
          if verbose == True:
            display(f'The frame {i} is identical to frame {i - 1}.')
          frame_test = False
        else:
          if verbose == True:
            display(f'Frame {i} is unique.')
          frame_test = True
        # display(f'{frame_count} greater than engarde length')

    # Saves the Image in Either Original or Original and Without Repeat
    # Excludes frames that are Part of the Engarde Positioning
    if (frame_test == True) or (i <= engarde_length) or (remove_duplicate_frames == False):
      # cv2.imwrite(name_orig, frame)
      # cv2.imwrite(name_orig_worpt, frame)

      # original_image_list.append(frame)
      original_image_worpt_list.append(original_image_list[i])

      if i > 0:
        camera_steady.append(average_diff)
      else:
        camera_steady.append(0)
    else:
      # cv2.imwrite(name_orig, frame)
      # original_image_list.append(frame)
      num_of_deleted_frames += 1

  return (camera_steady, duplicate_threshold, camera_motion_threshold, original_image_list, original_image_worpt_list)

In [ ]:
def setup_clip_directories():
  # Creates the Save, Original and Original without Repeats Folders

  os.chdir('/content/')

  if not exists('/content/Mask_RCNN'):
    os.mkdir('Mask_RCNN')

  os.chdir('/content/Mask_RCNN/')

  display(f'os.getcwd() is: {os.getcwd()}')
  ROOT_DIR = os.getcwd()
  MODEL_DIR = os.path.join(ROOT_DIR, "logs")
  VIDEO_DIR = os.path.join(ROOT_DIR, "videos")
  VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "save")
  VIDEO_ORIG_DIR = os.path.join(VIDEO_DIR, "original")
  VIDEO_ORIGWORPT_DIR = os.path.join(VIDEO_DIR, "original_without_repeats")
  display(f'The ROOT_DIR is: {ROOT_DIR}')

  # Removes and recreates the save directory effectively emptying the folder
  # Attempts to remove folders to ensure folders are empty
  try:
    shutil.rmtree('videos')
  except:
    display(f'The video directory did not exist.')
  os.mkdir('videos')
  os.chdir('/content/Mask_RCNN/videos')
  os.mkdir('save')
  os.mkdir('original')
  os.mkdir('original_without_repeats')


  return(ROOT_DIR, VIDEO_DIR, VIDEO_SAVE_DIR, VIDEO_ORIG_DIR)

In [ ]:
def run_human_pose(image, fencer_data, keypoints, already_run):
  # Runs Human Pose Analysis on an Image and returns fencer data and keypoints
  # Already Run ensures that it runs at most one time for a frame

  if already_run == False:
    # Uses Human Pose Analysis for Keypoints
    [fencer_data_temp, keypoints_temp] = human_pose_analysis(image)
    display(f'The fencer data is:')
    display(fencer_data)
    fencer_data.append(fencer_data_temp)
    keypoints.append(keypoints_temp)
    already_run = True

  return(fencer_data, keypoints, already_run)

In [ ]:
#@title Process_Video_Clip
def process_video_clip(file_name, touch_folder, remove_duplicate_frames):
  # Processes the video

  display(f'The video file_name is: {file_name}')

  # Initiates Conditions
  score_box_empty = False
  right_torso_empty = False
  left_torso_empty = False
  left_position_empty = False
  right_position_empty = False
  left_foot_empty = False
  right_foot_empty = False

  save_image_list = []
  Left_Position = []
  Right_Position = []
  Scoring_Box_Position = []
  Left_Torso_Position = []
  Right_Torso_Position = []
  Left_Foot_Position = []
  Right_Foot_Position = []
  left_torso_size_average = []
  right_torso_size_average = []
  Tracking_Bounding_Boxes = []
  All_Bell_Guard_Positions = []
  Exclusion_Areas = []
  bbox_clip = []

  skip_frames = False
  skip_frame_counter = 0
  frames_to_skip = 300
  number_of_frames_skipped = 0

  clip_vector_previous = []

  # Removes and Recreates Save, Original and Original without Repeats folders to ensure empty and moves the clip to the appropriate directory
  [ROOT_DIR, VIDEO_DIR, VIDEO_SAVE_DIR, VIDEO_ORIG_DIR] = setup_clip_directories()

  engarde_length = 10

  video_clip_dir = '/content/drive/My Drive/Sync/'
  [original_image_list, fps] = read_video(file_name, video_clip_dir)

  # Removes Duplicates and Detects Camera motion in Frames
  [camera_steady, duplicate_threshold, camera_motion_threshold, original_image_list, original_image_worpt_list] \
  = test_and_remove_duplicate_frames(original_image_list, engarde_length, remove_duplicate_frames)

  width = original_image_worpt_list[0].shape[1]
  height = original_image_worpt_list[0].shape[0]

  # Determines the Starting Frame of the file
  [starting_frame, total_frames] = determine_starting_frame(original_image_worpt_list, width, height)

  start_time = starting_frame/fps
  end_time = total_frames/fps

  orig_img_worpt_starting_list = original_image_worpt_list[starting_frame:]

  if total_frames == 0:
    display(f'ERROR: The Video Clip selected has no frames.')
  display(f'The total number of frames in the video are: {total_frames}')

  try:
    if not os.path.exists(VIDEO_SAVE_DIR):
          os.makedirs(VIDEO_SAVE_DIR)
  except OSError:
    print ('Error: Creating directory of data')

  frame_count = 0

  if verbose == True:
    display(f'The capture width is: {width}')
    display(f'The capture height is: {height}')
  
  # Continues until it breaks by not finding a return from attempting to read a frame capture

  # Creates a variable that allows looping if engarde positioning is not effective
  perform_engarde_positioning = True
  engarde_offset = 0
  # The number of frames that is incremented when an engarde positioning fails
  offset_increment = 10

  close_bellguards = False
  too_few_frames = False
  frames_after_light_iterator = 0
  frames_after_light_max = 15
  minimum_frames_to_run = 20

  if (total_frames - starting_frame) > minimum_frames_to_run:
    display(f'The remaining frames are {(total_frames - starting_frame)}, which is greater than {minimum_frames_to_run}.')
  else:
    display(f'The remaining frames are {(total_frames - starting_frame)}, which is fewer than {minimum_frames_to_run}.')

  # if ((total_frames - starting_frame) =< minimum_frames_to_run):
  if ((total_frames - starting_frame - engarde_offset) <= minimum_frames_to_run):
    display(f'Too Few Frames to run {file_name}')
    too_few_frames = True


  # Runs for 5 frames after the lights turn on and required at least 30 frames total
  while (total_frames - starting_frame) > minimum_frames_to_run and (frame_count) < len(orig_img_worpt_starting_list):
    # Iterator
    frame_count += 1
    #Creates Bounding Box List
    frame = orig_img_worpt_starting_list[frame_count - 1]
    frames = [frame]

    # Runs the Detection Model for Bellguard, Torso, Scoring_Box
    [bbox, frame] = yolov4_run_image(frame, True)
    save_image_list.append(frame)

    bbox_clip.append(bbox)

    if (frame_count <= (engarde_length + engarde_offset)):

      # Uses a frame in the middle of positioning to determine the time for frame analysis

      # Performs Engarde Positioning for a frame
      [Left_Position_Temp, Right_Position_Temp, Scoring_Box_Position_Temp, scoring_box_size_average, Tracking_Bounding_Boxes_Temp, \
        Left_Torso_Position_Temp, Right_Torso_Position_Temp, left_torso_size_average_Temp, right_torso_size_average_Temp, \
       All_Bell_Guard_Positions_Temp, Left_Foot_Position, Right_Foot_Position, Foot_Size] \
        = engarde_position(bbox, width, height, engarde_length + engarde_offset, frame_count-1, save_image_list)

      Left_Position.append(Left_Position_Temp)
      Right_Position.append(Right_Position_Temp)

      Scoring_Box_Position.append(Scoring_Box_Position_Temp)
      Left_Torso_Position.append(Left_Torso_Position_Temp)
      Right_Torso_Position.append(Right_Torso_Position_Temp)
      left_torso_size_average.append(left_torso_size_average_Temp)
      right_torso_size_average.append(right_torso_size_average_Temp)
      if verbose == True:
        display(f'At frame {frame_count} the tracking Bounding Boxes Temp is:')
        display(Tracking_Bounding_Boxes_Temp)
      Tracking_Bounding_Boxes.append(Tracking_Bounding_Boxes_Temp)
      # Creates a List of each Bell Guard Position in each engarde positioning frame. Used for excluding false positives
      All_Bell_Guard_Positions.append(All_Bell_Guard_Positions_Temp)

    if (frame_count == engarde_length + engarde_offset):

      if verbose == True:
        display(f'Commencing the Engarde Length Processing.')
      # tracked_items = [Left_Position, Right_Position, Scoring_Box_Position, Left_Torso_Position, Right_Torso_Position, left_torso_size_average, right_torso_size_average]
      tracked_items = [Left_Position, Right_Position, Scoring_Box_Position, Left_Torso_Position, Right_Torso_Position, left_torso_size_average, right_torso_size_average, Left_Foot_Position, Right_Foot_Position]

      if verbose == True:
        display(f'The Scoring Box Position is:')
        display(Scoring_Box_Position)
      if max(Scoring_Box_Position) == []:
        if verbose == True:
          display(f'The Scoring Box Position was empty so a default was used.')
        score_box_empty = True
        tracked_items[2] = [[width/2, height/2], [width/2, height/2], [width/2, height/2]]

      # Tests for empty Positions
      if max(Right_Torso_Position) == []:
        right_torso_empty = True

      if max(Left_Torso_Position) == []:
        left_torso_empty = True

      if max(Left_Position) == [] and max(Left_Torso_Position) != []:
        left_position_empty = True

      if max(Right_Position) == [] and max(Right_Torso_Position) != []:
        right_position_empty = True
      
      # Replaces [0,0] with [] for torso width and height
      for k in range(len(tracked_items)):
        for j in range(len(tracked_items[k])):
          if tracked_items[k][j] == [0,0]:
            tracked_items[k][j] = []

      for k in range(len(tracked_items)):
        if verbose == True:
          display(f'k is {k}.')
          display(tracked_items[k])
        try:
          tracked_items[k] = [item for item in tracked_items[k] if item != []]
          tracked_items[k] = np.hstack(tracked_items[k])
          if verbose == True:
            display(f'The length of len(tracked_items[k]) is {int(len(tracked_items[k])/2)}.')
          tracked_items[k] = tracked_items[k].reshape(int(len(tracked_items[k])/2), 2)
          tracked_items[k] = np.median(tracked_items[k], axis = 0)
          tracked_items[k] = [int(tracked_items[k][0]), int(tracked_items[k][1])]
          if verbose == True:
            display(f'The tracked item position {tracked_items[k]}.')
        except:
          if verbose == True:
            display(f'Failure to detect the tracked item {k} during the engarde positioning.')
            display(tracked_items[k])
          tracked_items[k] = [0,0]

      if right_torso_empty == True:
        # if verbose == True:
        display(f'The Right Torso Position was empty so a default based on the Right BellGuard was used.')
        display(f'tracked_items[1] is {tracked_items[1]}, left_torso_size_average is {left_torso_size_average}.')
        torso_position_default = [tracked_items[1][0] + int(left_torso_size_average[0][0]*3/4), tracked_items[1][1] - int(left_torso_size_average[0][1]/4)]
        tracked_items[4] = torso_position_default
        # Sets the Right Tracking Bounding Box
        # Sets the Right Torso Average Size to the Left Average Size
        right_torso_size_average = left_torso_size_average
        if verbose == True:
          display(f'As a comparison, the Score Box Position is:')
          display(Scoring_Box_Position)
          display(f'The Right Torso Position is:')
          display(Right_Torso_Position)
          display(f'As a comparison, the tracked_items[2] is:')
          display(tracked_items[2])
          display(f'The tracked_items[4] is:')
          display(tracked_items[4])
        # Sets Right Torso Empty to False after a Default it used
        right_torso_empty = False

      if left_torso_empty == True:
        if verbose == True:
          display(f'The Left Torso Position was empty so a default based on the Left BellGuard was used.')
          display(f'tracked_items[0] is {tracked_items[0]}, right_torso_size_average is {right_torso_size_average}.')
        torso_position_default = [tracked_items[0][0] - int(right_torso_size_average[1][0]*3/4), tracked_items[0][1] - int(right_torso_size_average[0][1]/4)]
        tracked_items[3] = torso_position_default
        # Sets the Left Torso Average Size to the Left Average Size
        left_torso_size_average = right_torso_size_average
        if verbose == True:
          display(f'As a comparison, the Score Box Position is:')
          display(Scoring_Box_Position)
          display(f'The Left Torso Position is:')
          display(Left_Torso_Position)
          display(f'As a comparison, the tracked_items[1] is:')
          display(tracked_items[1])
          display(f'The tracked_items[3] is:')
          display(tracked_items[3])
        left_torso_empty = False

      if left_position_empty == True:
        if verbose == True:
          display(f'The Left  Position was empty so a default based on the Left Torso was used.')
        left_torso_size_average_temp = average_list_without_null(left_torso_size_average)
        left_position_default = [int(tracked_items[3][0] + left_torso_size_average_temp[0]), int(tracked_items[3][1] + left_torso_size_average_temp[0]/4)]
        # left_position_default = [tracked_items[3][0] + int(left_torso_size_average[0][0]), tracked_items[3][1] + int(left_torso_size_average[0][0]/4)]
        tracked_items[0] = left_position_default

      if right_position_empty == True:
        if verbose == True:
          display(f'The Right  Position was empty so a default based on the Right Torso was used.')
          display(f'The tracked_items[4] is {tracked_items[4]}.')
          display(f'The right_torso_size_average is {right_torso_size_average}.')
        right_torso_size_average_temp = average_list_without_null(right_torso_size_average)
        right_position_default = [int(tracked_items[4][0] - right_torso_size_average_temp[0]), int(tracked_items[4][1] + right_torso_size_average_temp[0]/4)]
        tracked_items[1] = right_position_default

      [Left_Position, Right_Position, Scoring_Box_Position, Left_Torso_Position, Right_Torso_Position, Left_Foot_Position, Right_Foot_Position] = [[],[],[],[],[],[],[]]

      if verbose == True:
        display(f'The length of the tracked items is {len(tracked_items)}')

      # Builds the Positions of the Tracked Items, iterator is solely to append multiple times
      for i in range(engarde_length + engarde_offset):
        Left_Position.append(tracked_items[0])
        Right_Position.append(tracked_items[1])
        Scoring_Box_Position.append(tracked_items[2])
        Left_Torso_Position.append(tracked_items[3])
        Right_Torso_Position.append(tracked_items[4])
        Left_Foot_Position.append(tracked_items[7])
        Right_Foot_Position.append(tracked_items[8])

      if verbose == True:
        display(f'The Left Foot Position is:')
        display(Left_Foot_Position)

      padding = width/24
      exclusion_box = [padding,padding,padding,padding]
      Left_Exculsion_Box = create_boundary_box(Left_Position[0],exclusion_box, False)
      Right_Exculsion_Box = create_boundary_box(Right_Position[0],exclusion_box, False)

      for m in range(len(All_Bell_Guard_Positions)):
        for n in range(len(All_Bell_Guard_Positions[m])):
          within_Left_Region = boundary_box_test(All_Bell_Guard_Positions[m][n], Left_Exculsion_Box)
          within_Right_Region = boundary_box_test(All_Bell_Guard_Positions[m][n], Right_Exculsion_Box)
          if within_Left_Region == False and within_Right_Region == False:
            Exclusion_Areas.append(All_Bell_Guard_Positions[m][n])

      # Removes duplicate Exclusion Areas that are within a given distance of each other
      Exclusion_Areas = exclusion_area_simplification_recursion(Exclusion_Areas, width/80)

      if verbose == True:
        display(f'The Exclusion areas are:')
        display(Exclusion_Areas)

      left_torso_size_average = tracked_items[5]
      if right_torso_empty == False:
        right_torso_size_average = tracked_items[6]
      elif right_torso_empty == True:
        right_torso_size_average = tracked_items[5]
      torso_size = [left_torso_size_average, right_torso_size_average]

      if verbose == True:
        display(f'The left_torso_size_average is {left_torso_size_average} and the right_torso_size_average is {right_torso_size_average}.')
        display(f'The Right_Torso_Position[0] Prior to testing is: {Right_Torso_Position[0]}. ')

      if verbose == True:
        display(f'Prior to testing for success, the Left Position[0] is:')
        display(Left_Position[0])
        display(f'Prior to testing for success, the Right Position[0] is:')
        display(Right_Position[0])
        display(f'Prior to testing for success, the Left_Torso_Position[0] is:')
        display(Left_Torso_Position[0])
        display(f'Prior to testing for success, the Right_Torso_Position[0] is:')
        display(Right_Torso_Position[0])
        display(f'Perform_engarde_positioning is {perform_engarde_positioning}.')

      if (Left_Position[0] == [0,0] or Right_Position[0] == [0,0] or Left_Torso_Position[0] == [0,0] or Right_Torso_Position[0] == [0,0]) and (perform_engarde_positioning == True):
      # if max(Left_Position) == [] or max(Right_Position) == [] or max(Left_Torso_Position) == [] or max(Right_Torso_Position) == []:
        # perform_engarde_positioning = True
        engarde_offset = engarde_offset + offset_increment
        display(f'The Engarde Positioning Failed, incrementing the Engarde Length by {offset_increment} frames.')
        if verbose == True:
          display(f'The Left Torso Position at the end of Engarde Positioning is: {Left_Torso_Position}.')
          display(f'The Right Torso Position at the end of Engarde Positioning is: {Right_Torso_Position}.')
      else:
        perform_engarde_positioning = False
        display(f'The Engarde Positioning was successful, continuing with clip analysis.')
        display(f'The final engarde length was {engarde_length + engarde_offset}.')
        if verbose == True:
          display(f'The Left Torso Position at the end of Engarde Positioning is: {Left_Torso_Position}.')
          display(f'The Right Torso Position at the end of Engarde Positioning is: {Right_Torso_Position}.')

    if verbose == True:
      display(f'The frame count is {frame_count} and perform engarde positioning is {perform_engarde_positioning}.')
    # Continues on the frames beyond the engarde length if positioning was successful
    if verbose == True:
      display(f'The frame count is {frame_count} and the engarde length is {engarde_length} with an offset of {engarde_offset}. Positioning is {perform_engarde_positioning}.')
    
    
    # display(f'The frame_count is {frame_count}, the engarde_length is {engarde_length} and engarde_offset is {engarde_offset}.')
    if (frame_count > (engarde_length + engarde_offset)) and (perform_engarde_positioning == False):

      positions = [Left_Position, Right_Position, Scoring_Box_Position, Left_Torso_Position, Right_Torso_Position, Left_Foot_Position, Right_Foot_Position]

      # Sets the certainty following the engarde positioning
      if frame_count == engarde_length + engarde_offset + 1:
        t3 = time.time()
        certainty = [0,0,0,0,0,0,0]
        if verbose == True:
          display(f'The positions following the engarde positioning are:')
          display(positions)

      if verbose == True:
        display(f'Certainty just prior to Bell Guard Positioning is {certainty}.')

      previous_certainty = certainty

      if right_torso_size_average[0] == 0 and left_torso_size_average[0] == 0:
        if verbose == True:
          display(f'Error, both Torso sizes are zero.')   
      elif right_torso_size_average[0] == 0:
        if verbose == True:
          display(f'Right Torso Size was zero, using the Left as a Defualt.')
        right_torso_size_average = left_torso_size_average
      elif left_torso_size_average[0] == 0:
        if verbose == True:
          display(f'Left Torso Size was zero, using the Right as a Defualt.')
        left_torso_size_average = right_torso_size_average

      if verbose == True:
        display(f'The engarde_length is {engarde_length}.')
        display(f'The engarde_offset is {engarde_offset}.')

      if verbose == True:
        display(f'Right Torso Size average is {right_torso_size_average} at frame {frame_count} just prior to Bell Guard Positioning.')
        display(f'Right Torso Position is {positions[4]} at frame {frame_count}.')

      # Finds the Tracked Items and Returns their positions
      [Left_Position_Temp, Right_Position_Temp, Scoring_Box_Position_Temp, Tracking_Bounding_Boxes_Temp, \
      Left_Torso_Position_Temp, Right_Torso_Position_Temp, Left_Foot_Position_Temp, Right_Foot_Position_Temp, certainty] \
      = Bell_Guard_Position_Finding(bbox, width, height, positions, frame_count, \
      left_torso_size_average, right_torso_size_average, (engarde_length + engarde_offset), \
      certainty, camera_steady, camera_motion_threshold, Exclusion_Areas, orig_img_worpt_starting_list)

      if verbose == True:
        display(f'Certainty just after to Bell Guard Positioning is {certainty}.')
        display(f'The Left Position at frame {frame_count - 1} is {Left_Position_Temp}.')

      if close_bellguards == True:
        frames_after_light_iterator = frames_after_light_iterator + 1

      # If the Positions are Close enough then the lights are assumed on and tracking is stopped
      if ((Right_Position_Temp[0] - Left_Position_Temp[0]) < width*position_difference_ratio) or (close_bellguards == True):
        close_bellguards = True
        # display(f'close_bellguards set to True')
        if frames_after_light_iterator == 0:
          display(f'The last tracked frame is {frame_count}.')
          # Adjusts the frames after light to include the entire clip
          if run_entire_video == True:
            frames_after_light_max = total_frames - frame_count
          # Ensures that frames processed does not exceed total number of frames
          # if (frames_after_light_iterator + frame_count) >= total_frames:
          if (frames_after_light_max + frame_count + starting_frame) >= total_frames:
          # if (frames_after_light_max + frame_count + engarde_offset) >= total_frames:
            # frames_after_light_max = (frames_after_light_iterator + frame_count - 1)
            # frames_after_light_max = total_frames - (frames_after_light_max + frame_count)
            frames_after_light_max = total_frames - (frames_after_light_max + frame_count + starting_frame)
            # frames_after_light_max = total_frames - (frames_after_light_max + frame_count + engarde_offset)
            display(f'The frames after light max has been adjusted to {frames_after_light_max}')

      # Appends the Returned Positions
      Left_Position.append(Left_Position_Temp)
      Right_Position.append(Right_Position_Temp)
      Scoring_Box_Position.append(Scoring_Box_Position_Temp)
      Left_Torso_Position.append(Left_Torso_Position_Temp)
      Right_Torso_Position.append(Right_Torso_Position_Temp)
      Left_Foot_Position.append(Left_Foot_Position_Temp)
      Right_Foot_Position.append(Right_Foot_Position_Temp)
      Tracking_Bounding_Boxes.append(Tracking_Bounding_Boxes_Temp)

      # Tests for a change in certainty to zero from non-zero. If a position has become certain during
      # this frame then it back calculates previous uncertain position up to a certain position.
      if (certainty[0] == 0 and previous_certainty[0] != 0):
        Left_Position = position_linear_approximation(Left_Position, previous_certainty[0])
        if verbose == True:
          display(f'Using a Linear Approximation for frame {frame_count} for the Left Bellguard Position.')
      elif (certainty[1] == 0 and previous_certainty[1] != 0):
        if verbose == True:
          display(f'The Right Position is: {Right_Position}.')
          display(f'The Previous Certainty is: {previous_certainty[1]}')
          display(f'Using a Linear Approximation for frame {frame_count} for the Right Bellguard Position.')
        Right_Position = position_linear_approximation(Right_Position, previous_certainty[1])
      elif (certainty[2] == 0 and previous_certainty[2] != 0):
        Scoring_Box_Position = position_linear_approximation(Scoring_Box_Position, previous_certainty[2])
      elif (certainty[3] == 0 and previous_certainty[3] != 0):
        Left_Torso_Position = position_linear_approximation(Left_Torso_Position, previous_certainty[3])
      elif (certainty[4] == 0 and previous_certainty[4] != 0):
        Right_Torso_Position = position_linear_approximation(Right_Torso_Position, previous_certainty[4])
      elif (certainty[5] == 0 and previous_certainty[5] != 0):
        Left_Foot_Position = position_linear_approximation(Left_Foot_Position, previous_certainty[5])
      elif (certainty[6] == 0 and previous_certainty[6] != 0):
        Right_Foot_Position = position_linear_approximation(Right_Foot_Position, previous_certainty[6])
      else:
        pass    

  # End of Individual Frame Processing

  if not too_few_frames:

    if verbose == True:
      # Reduces the Frame Count to account for skipped frames
      display(f'The original frame count was: {frame_count - 1} and the number of frames skipped is: {number_of_frames_skipped}.')
    frame_count = len(bbox)
    if verbose == True:
      display(f'The length of the frame_count is {frame_count - 1} while the number of bboxes is {len(bbox)}.')

    file_to_remove = r'/Mask_RCNN/videos/' + file_name
    try:
      shutil.rmtree(file_to_remove)
    except:
      if verbose == True:
        display(f'ERROR removing the video file to analyze.')

    if verbose == True:
      display(f'The Left Position just prior to drawing the Bell_Guards is:')
      display(Left_Position)
      display(f'The Right Position just prior to drawing the Bell_Guards is:')
      display(Right_Position)
      display(f'The Score Box Position just prior to drawing the Bell_Guards is:')
      display(Scoring_Box_Position)

    # Allows for a simple clip vector to draw in the lights if an overlay is used instead of a representative clip
    simple_clip_vector = simple_clip_vector_generator(Left_Position, Right_Position, width)

    if verbose == True:
      display(f'The Right Torso ')
      display(f'The Right Torso Position Just Prior to Drawing at frame {frame_count} is: {Right_Torso_Position}')

    #Draws the Boxes on the image frame and determines scoring lights turned on
    [left_light_comparison, right_light_comparison, save_image_list] = \
    draw_Bell_Guard_Position(Left_Position, Right_Position, Scoring_Box_Position, scoring_box_size_average, \
    Left_Torso_Position, Right_Torso_Position, Left_Foot_Position, Right_Foot_Position, frame_count, Tracking_Bounding_Boxes, \
    video_filename, width, height, engarde_length + engarde_offset, score_box_empty, camera_steady, camera_motion_threshold, \
    Exclusion_Areas, simple_clip_vector, save_image_list)


    if camera_motion_compensate == True and score_box_empty == False:
      #Adjusts the Bellguard Position Based on the Camera motion as determined by the Score_Box Position
      Left_Position = camera_motion_adjustment(Left_Position, Scoring_Box_Position)
      Right_Position = camera_motion_adjustment(Right_Position, Scoring_Box_Position)

    if camera_motion_compensate == True and score_box_empty == False:
      #Adjusts Left and Right Position for convenient visualization
      [Left_Position, Right_Position] = position_down_scale(Left_Position, Right_Position, width, height)

    clip_vector = clip_vector_generator(Left_Position, Right_Position, left_light_comparison, right_light_comparison, clip_vector_previous, width)

    if smooth_video_clip == True:
      #Smoothes the Clip using Savitzky–Golay filter
      clip_vector = smooth_clip_vector(clip_vector, engarde_length + engarde_offset)

    if verbose == True:
      display(f'The final clip vector is:')
      display(clip_vector)

    # clip_vector_np_save(touch_folder, file_name[:-4], clip_vector, file_title)
    clip_vector_np_save(touch_folder, file_name[:-4], clip_vector, file)

  else:
    # Sets output variables to none since the clip cannot run due to too few frames
    bbox = clip_vector = 'none'

  return (bbox_clip, frame_count, width, height, clip_vector_previous, clip_vector, fps, save_image_list, too_few_frames)

In [ ]:
#@title Prepare_Video_Frames
def prepare_video_frames(image_save_directory, video_directory, video_name):
  VIDEO_DIR = video_directory
  VIDEO_SAVE_DIR = image_save_directory
  images = list(glob.iglob(os.path.join(VIDEO_SAVE_DIR, '*.*')))
  # Sort the images by integer index
  images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))

  name = str(video_name) + '.mp4'
  outvid = os.path.join(VIDEO_DIR, name)

  return (outvid, images)

In [ ]:
#@title Downsample_FPS
def downsample_fps(a,b):
  # Adjusts the elements of a larger set a to fit into the length of set b

  c = []
  remainder = 0
  for i in range(len(b)):
    c_temp = []
    if verbose == True:
      display(f'The lower range is {math.ceil(len(a)/len(b)*(i+1)-1-remainder)} and the upper range is {math.floor(len(a)/len(b)*(i+1))}.')
    for j in range(math.ceil(len(a)/len(b)*(i)-remainder),math.floor(len(a)/len(b)*(i+1))):
      c_temp.append(a[j])
      if verbose == True:
        display(f'i,j = {i},{j} and c_temp = {c_temp}')
    remainder = (len(a)/len(b))*(i+1) - int(len(a)/len(b)*(i+1))
    c.append(round(sum(c_temp)/len(c_temp)))
    if verbose == True:
      display(f'The remainder at i = {i} and j = {j} is {remainder} and c is {c}.')

  return (c)

In [ ]:
#@title Load_Clip
def load_clip(folder, file_name, capture_width):
  if folder == 'Left' or folder == 'left' or folder == 'Left_Touch':
    folder = 0
  if folder == 'Right' or folder == 'right'or folder == 'Right_Touch':
    folder = 1
  if folder == 'Simul' or folder == 'simul'or folder == 'Simul':
    folder = 2

  engarde_position_buffer = 15
  max_length = 111
  clip_vector_length = max_length - engarde_position_buffer


  touch_folder = ['Left_Touch', 'Right_Touch', 'Simul']

  i = folder

  # file = 'clip_vector_acceleration_np' + str(clip_number) + '.csv'
  file = file_name
  path = folder
  # path = r'/content/drive/My Drive/projects/fencing/Fencing Clips/' + touch_folder[i] + '/' + touch_folder[i] + '_Vector_Clips_Acceleration/'

  vector_data = pd.read_csv(os.path.join(path, file), header=None)
  clip_vector = vector_data.to_numpy(dtype = np.float32)

  display(os.path.join(path, file))

  # Pads the clip_vector to max_length
  # If the clip is greater than Max Length, it is truncated
  if len(clip_vector) > max_length:
    clip_vector = clip_vector[len(clip_vector) - max_length:]
  padding = np.array([0,0,0,0])
  for k in range(max_length - (len(clip_vector))):
    clip_vector = np.vstack((clip_vector, padding))

  #Normalizes the Values
  max_value = int(capture_width/42)
  for i in range(len(clip_vector)):
    for j in range(2):
      if clip_vector[i][j] < max_value:
        clip_vector[i][j] = clip_vector[i][j] * (1/max_value)
      else:
        #Preserves the sign of the value
        clip_vector[i][j] = clip_vector[i][j]/(abs(clip_vector[i][j]))

  # Removes the First 15 frames to minimize engarde positioning
  clip_vector = clip_vector[15:]

  # Sets Clip_Vector to Zero if Light is on
  for j in range(len(clip_vector)):
    if clip_vector[j][2] == 1:
      clip_vector[j][0] = 0
    if clip_vector[j][3] == 1:
      clip_vector[j][1] = 0 

  clip_vector = clip_vector.reshape(1,clip_vector_length,4)
  return (clip_vector)

In [ ]:
#@title Create_Folder_Hierarchy
def create_folder_hierarchy(file_name, output_bbox_clip, output_foot_data):

  # Changes the directory to the top level of My Drive
  if not exists('/content/drive/My Drive/Sync/Acceleration'):
    os.makedirs('/content/drive/My Drive/Sync/Acceleration')
    display(f'Created the Sync Acceleration directory.')
  else:
    display(f'The Sync Acceleration directory already exists.')
  # Creates the Tracked Clips Folder
  if not exists('/content/drive/My Drive/Sync/Tracked Clips'):
    os.makedirs('/content/drive/My Drive/Sync/Tracked Clips')
    display(f'Created the Sync Tracked Clips directory.')
  else:
    display(f'The Sync Tracked Clips directory already exists.')

  # Creates the Boundary Box Outputs if required
  if output_bbox_clip:
    if output_foot_data:
      if not exists('/content/drive/My Drive/Sync/Foot Data'):
        os.makedirs('/content/drive/My Drive/Sync/Foot Data')
        display(f'Created the Foot Data directory.')
      else:
        display(f'The Foot Data directory already exists.')

  return ()

In [ ]:
#@title Evaluate_Model
def evaluate_model(dataset, model, cfg):
  # calculate the mAP for a model on a given dataset
  APs = list()
  for image_id in dataset.image_ids:
		# load image, bounding boxes and masks for the image id
    image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
		# convert pixel values (e.g. center)
    scaled_image = mold_image(image, cfg)
		# convert image into one sample
    sample = expand_dims(scaled_image, 0)
		# make prediction
    yhat = model_detection.detect(sample, verbose=0)
		# extract results for first sample
    r = yhat[0]
		# calculate statistics, including AP
    AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
		# store
    APs.append(AP)
	# calculate the mean AP across all images
  mAP = mean(APs)
  return mAP

In [ ]:
def youtube_clip(link):

  os.chdir('/content/')

  # Removes Potential Previous Files
  if exists('video_title.mp4'):
    os.remove('video_title.mp4')
  if exists('video_title.mkv'):
    os.remove('video_title.mkv')

  # Gets the Link for the Youtube Clip
  youtube_clip = link.split('=')[0][0:-2]
  # Uses the YouTube Query to get the time in seconds of the position in the clip
  youtube_clip_position = int(link.split('=')[1])
  # Clip End Position
  youtube_end_position = int(youtube_clip_position + 10)
  
  ydl_opts = {}
  #Gets info for the YouTube clip
  with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(link, download=False)
    video_title = info_dict.get('title', None)
    file_size = info_dict.get('formats',None)[0]['filesize']
    duration = info_dict.get('duration',None)
    # container = info_dict.get('formats',None)[0]['container']

  container = 0

  #Determines where to save the YouTube clip
  # path = f'./{video_title}.mp4'
  path = f'./video_title.mp4'
  ydl_opts.update({'outtmpl':path})

  #Downloads and saves the YouTube clip
  with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([link])

  # The video download may be .mp4 or mkv
  if exists('video_title.mp4'):
    video_title = 'video_title.mp4'
  if exists('video_title.mkv'):
    video_title = 'video_title.mkv'

  if verbose == True:
    display(f'The clip duration is {duration}.')

  if container == 'm4a_dash':
    display(f'The youtube link is a streamed video, downloading entire stream.')

  file_name = '999.mp4'
  touch_folder = 'Left_Touch'

  # Downloads the entire file if the video clip is short or a DASH file, i.e. streamed video clip
  if duration < duration_download_limit or container == 'm4a_dash':
    display(f'Duration is less than {duration_download_limit}. Downloading entire clip.')
    !ffmpeg -y -ss $youtube_clip_position -i $video_title -c:v libx264 -c:a aac -frames:v 100 999.mp4
  else:
    display(f'Duration is greater than {duration_download_limit}. Downloading a portion of the clip.')
    !ffmpeg $(youtube-dl -g $link | sed "s/.*/-ss $youtube_clip_position -i &/") -y -to $youtube_end_position -c copy out.mp4
    [starting_frame, fps, total_frames] = determine_starting_frame('out.mp4', touch_folder, True)
    os.chdir('/content/')
    frames_per_second = 25
    start_time = starting_frame/frames_per_second
    # Clips the Video per the starting frame
    !ffmpeg -y -ss $start_time -i out.mp4 -c:v libx264 -c:a aac -frames:v 125 999.mp4
    # Copies the video to the appropriate folder
    destination = r'/content/drive/My Drive/projects/fencing/Fencing Clips/Left_Touch/'
    file_name_with_path = os.path.join(destination, file_name)
    #Copies the clip to the Left Touch Folder. Tests for file already existing
    if exists(file_name_with_path):
      os.remove(file_name_with_path)
    shutil.copy(file_name, destination)

  destination = r'/content/drive/My Drive/projects/fencing/Fencing Clips/Left_Touch/'
  file_name_with_path = os.path.join(destination, file_name)
  #Copies the clip to the Left Touch Folder. Tests for file already existing
  if exists(file_name_with_path):
    os.remove(file_name_with_path)
  shutil.copy(file_name, destination)

  return

In [ ]:
def determine_starting_frame(original_image_worpt_list, width, height):
  # Determines the first usable frame in a clip

  # Sets up the clip directories
  [ROOT_DIR, VIDEO_DIR, VIDEO_SAVE_DIR, VIDEO_ORIG_DIR] = setup_clip_directories()

  os.chdir('/content/drive/My Drive/')  

  total_frames = len(original_image_worpt_list)
  if total_frames == 0:
    display(f'ERROR: The Video Clip selected has no frames.')
  display(f'The total number of frames in the video are: {total_frames}')

  # frame_check keeps a running account of detected object in each frame
  frame_check = []
  # Determines when the first complete frame of a clip is found
  test_frame_check = False
  # Initialize the frame iterator, counts the number of frames analyzed
  k = 0
  # Number of Frames to Sample
  group_size = 10
  # Number of Frames that must be true
  min_positives = 2
  # min_positives = 1

  while test_frame_check == False and k < len(original_image_worpt_list):

    # display(f'k is {k} and the length of orig_worpt is {len(original_image_worpt_list)}.')

    frame = original_image_worpt_list[k]
    # Runs the Detection Model
    [bbox, none] = yolov4_run_image(frame, False)

    if bbox != []:

      if verbose == True:
        display(f'a is:')
        display(a)

      # Tests if there is a viable Left and Right Torso in the Frame
      [Left_Torso_Detection, Left_Torso_Box_Max] = torso_detection_test(bbox, 'Left', width, height)
      [Right_Torso_Detection, Right_Torso_Box_Max]  = torso_detection_test(bbox, 'Right', width, height)

      # Tests if there is a viable Left and Right Bell Guard in the Frame
      Left_Bell_Guard_Detection = bell_guard_detection_test(bbox, Left_Torso_Box_Max, 'Left', width, height)
      Right_Bell_Guard_Detection  = bell_guard_detection_test(bbox, Right_Torso_Box_Max, 'Right', width, height)

      # display(f'At frame {m} the L/R Torso is {Left_Torso_Detection}/{Right_Torso_Detection} and L/R Bellguard is {Left_Bell_Guard_Detection}/{Right_Bell_Guard_Detection}. ')

      # Checks if all the detections are True and appends 1 or 0 to the frame_check.
      if Left_Bell_Guard_Detection == Right_Bell_Guard_Detection == Left_Torso_Detection == Right_Torso_Detection == True:
        frame_check.append(1)
      else:
        frame_check.append(0)

    # Increments the frame iterator
    k = k + 1

    # Tests the frame_check to determine if there are sufficient positives
    if len(frame_check) > group_size:
      sum_of_values = 0
      for i in range(group_size):
        sum_of_values = sum_of_values + frame_check[len(frame_check) - i - 1]
  
      if verbose == True:
        display(f'The number of positives is {sum_of_values}.')
      if sum_of_values >= min_positives:
        if verbose == True:
          display(f'The threshold has been met. Success.')
        test_frame_check = True
      else:
        if verbose == True:
          display(f'The minimum has not been achieved. Failure.')

  starting_frame = k - group_size

  return (starting_frame, total_frames)

In [ ]:
def bell_guard_detection_test(bbox, Torso_Box_Max, side, width, height):

  # Initializes the detection variable
  detection = False

  y_min = int(height*1/6)
  y_max = min(int(height*2/3), Torso_Box_Max)
  if side == 'Left':
    x_min = 0
    x_max = width*1/2
  elif side == 'Right':
    x_min = width*1/2
    x_max = width

  for i in range(len(bbox)):
    # Defines the Center of the BBox
    x_avg = int((bbox[i][0][1]+bbox[i][0][3])/2)
    y_avg = int((bbox[i][0][0]+bbox[i][0][2])/2)

    if verbose == True:
      display(f'bbox[i] is {bbox[i]}.')
      display(f'bbox[i][2] is {bbox[i][2]}.')

    # Checks if the BBox is within the Bounds for plausible Bell Guard
    if  bbox[i][2] == 1 and x_min < x_avg and x_avg < x_max and y_min < y_avg and y_avg < y_max:
      detection = True
      break

  return (detection)

In [ ]:
def torso_detection_test(bbox, side, width, height):

  # Initializes the detection variable
  detection = False

  y_min = int(height*1/6)
  y_max = int(height*2/3)
  if side == 'Left':
    x_min = 0
    x_max = width*1/2
  elif side == 'Right':
    x_min = width*1/2
    x_max = width

  for i in range(len(bbox)):
    # Defines the Center of the BBox
    x_avg = int((bbox[i][0][1]+bbox[i][0][3])/2)
    y_avg = int((bbox[i][0][0]+bbox[i][0][2])/2)
    # Used in Bell Guard to verify bellguard is above the bottom of the Torso Detection
    y_box_max = int(bbox[i][0][2])
    # Checks if the BBox is within the Bounds for plausible Bell Guard
    if  bbox[i][2] == 3 and x_min < x_avg and x_avg < x_max and y_min < y_avg and y_avg < y_max:
      detection = True
      break

  return (detection, y_box_max)

In [ ]:
def Most_Recent_Sync_File():

  # Creates a List of Modified File Names and Dates modifies and selects the most recent.
  # Initializes the Lists
  date_modified_list = []
  file_size_list = []
  file_names = []
  path = '/content/drive/My Drive/Sync/'

  # Cycles through the files to generate the lists
  for file in os.listdir(path):
    # Ensures that the File is not a Folder
    if os.path.isfile(os.path.join(path, file)):
      name = os.path.join(path, file)
      file_names.append(file)
      # date_modified_list.append(os.path.getmtime(name))
      date_modified_list.append(os.path.getctime(name))
      file_size_list.append(int(os.path.getsize(name)/1024))
      # display(file)

  if verbose == True:
    display(max(date_modified_list))
  # Finds the index of the most recent file name
  if date_modified_list != []:
    index_of_max = date_modified_list.index(max(date_modified_list))
    if verbose == True:
      display(f'The most recent file is {file_names[index_of_max]}.')

    clip = file_names[index_of_max]
    size = file_size_list[index_of_max]
  else:
    clip = 'None'
    size = 'None'

  return (clip, size)

In [ ]:
def Move_Sync_File_for_Analysis(file_name):

  # Removes existing 999 file, copies and changes name of most recent file to Left Touch Folder
  # destination = '/content/drive/My Drive/projects/fencing/Fencing Clips/Left_Touch/'

  
  if not exists('/content/Mask_RCNN/videos'):
    os.mkdir('/content/Mask_RCNN/videos')

  destination = '/content/Mask_RCNN/videos'
  # file_name = clip
  file_source = '/content/drive/My Drive/Sync/'
  file_name_with_source = os.path.join(file_source, file_name)
  file_name_with_destination = os.path.join(destination, '999.mp4')
  if exists(file_name_with_destination):
    os.remove(file_name_with_destination)
    if verbose == True:
      f'{file_name} was removed from the Left Touch Folder.'
  else:
    if verbose == True:
      f'There was no file to remove from the Left Touch Folder.'
  shutil.copy(file_name_with_source, file_name_with_destination)

  if exists(file_name_with_destination):
    display(f'The file {file_name} has been moved to {destination}.')
  else:
    display(f'The file {file_name} was not successfully moved.')

  return

In [ ]:
def save_ffmpeg(source, destination, name, fps):

  os.chdir(source)
  file_name_with_destination = os.path.join(destination, name)

  if verbose == True:
    display(f'The source of the files is {source}.')
    display(f'The save destination is {destination}.')
    display(f'Name with destionation is {file_name_with_destination}.')
    display(f'The name is {name}.')

  !ffmpeg -framerate $fps -i %1d.jpg output.mp4

  file_name_with_destination = os.path.join(destination, name)
  name = name + '.mp4'

  shutil.copy('output.mp4', file_name_with_destination)

  return

In [ ]:
def darknet_helper(img, width, height):
  # darknet helper function to run detection on image

  darknet_image = make_image(width, height, 3)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_resized = cv2.resize(img_rgb, (width, height),
                              interpolation=cv2.INTER_LINEAR)

  # get image ratios to convert bounding boxes to proper size
  img_height, img_width, _ = img.shape
  width_ratio = img_width/width
  height_ratio = img_height/height

  # run model on darknet style image to get detections
  copy_image_from_bytes(darknet_image, img_resized.tobytes())
  detections = detect_image(network, class_names, darknet_image)
  free_image(darknet_image)

  return (detections, width_ratio, height_ratio)

In [ ]:
def yolov4_run_image(image, return_image):

  # %cd '/content/darknet/'

  os.chdir('/content/darknet/')

  # run test on person.jpg image that comes with repository

  detections, width_ratio, height_ratio = darknet_helper(image, width, height)

  bbox_temp = []
  names_of_objects = ['BG', 'bellguard', 'scorebox', 'torso', 'foot']

  if return_image:
    for label, confidence, bbox in detections:
      left, top, right, bottom = bbox2points(bbox)
      left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
      cv2.rectangle(image, (left, top), (right, bottom), class_colors[label], 2)
      cv2.putText(image, "{} [{:.2f}]".format(label, float(confidence)),
                        (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        class_colors[label], 2)
  else:
    image = 'None'

  for label, confidence, bbox in detections:
    left, top, right, bottom = bbox2points(bbox)
    left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
    arr = np.array([top, left, bottom, right])

    # change the dtype to 'float64' 
    arr = arr.astype('int32')
    percentage = float(math.floor(float(confidence))/100)
    class_id = names_of_objects.index(label)

    bbox_temp.append([arr, percentage, class_id])

  return(bbox_temp, image)

In [ ]:
def number_of_files_in_dir(path):
  files = [i for i in os.listdir(path)]
  number_of_files = len(files)

  return (number_of_files)

In [ ]:
def bbox_output(video_name, bbox_clip, output_foot_data):
  # Creates a CSV file of each detection of the format:
  # [frame, x_center, y_center, detection confidence]

  tracked_item = 0

  if output_foot_data == True:
    tracked_item = 4

  if tracked_item != 0:
    clip_items = []

    for i in range(len(bbox_clip)):
      for j in range(len(bbox_clip[i])):
        if bbox_clip[i][j][2] == tracked_item:
          x_center = int((bbox_clip[i][j][0][3]+bbox_clip[i][j][0][1])/2)
          y_center = int((bbox_clip[i][j][0][2]+bbox_clip[i][j][0][0])/2)
          clip_items.append([i,x_center,y_center, bbox_clip[i][j][1]])

    df = pd.DataFrame(clip_items, columns=['frame','x', 'y', 'confidence'])
    %cd '/content/drive/My Drive/Sync/Foot Data'
    name = video_name[:-4] + '_foot.csv'
    df.to_csv(name, index=False, header = True)


  return ()

In [ ]:
# Checks if the Folder Hierarchy is in place

display(f'Checking Folder Hierarchy...')
create_folder_hierarchy(video_filename, output_bbox_clip, output_foot_data)
# if not exists('/content/drive/My Drive/projects/fencing/Fencing Clips/Left_Touch/Left_Touch_Vector_Clips'):
#   create_folder_hierarchy(video_filename)
# else:
#   display(f'The Folder Hierarchy is already in place.')

ROW_Model_in_use = 'ROW_model.h5'

# Checks if the ROW model is in the Fencing Clip Folder or the top level
if exists('/content/drive/My Drive/' + ROW_Model_in_use):
  display(f'Loading the Right of Way Model from the top level directory.')
  ROW_Model_save_path = '/content/drive/My Drive/'
elif exists('/content/drive/My Drive/projects/fencing/Fencing Clips/ROW_model.h5'):
  display(f'Loading the Right of Way Model from Fencing Clips directory.')
  ROW_Model_save_path = '/content/drive/My Drive/projects/fencing/Fencing Clips/'
else:
  # wget tool
  # https://angelov.ai/post/2020/wget-files-from-gdrive/
  # Loads the ROW_Model from a shared file
  display(f'Loading the ROW_Model into Content')
  %cd '/content/'
  # !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1nvgqSTFITW74gPhbYX0hFZAmsN_MSBd_' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1nvgqSTFITW74gPhbYX0hFZAmsN_MSBd_" -O ROW_model.h5 && rm -rf /tmp/cookies.txt
  !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1xy_yvOmmzHkVzS_voJAXninjFTDFVdLn' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1xy_yvOmmzHkVzS_voJAXninjFTDFVdLn" -O ROW_model.h5 && rm -rf /tmp/cookies.txt
  source = 'ROW_model.h5'
  ROW_Model_save_path = '/content/'

'Checking Folder Hierarchy...'

'Created the Sync Acceleration directory.'

'Created the Sync Tracked Clips directory.'

'Loading the Right of Way Model from the top level directory.'

In [ ]:
loop_analysis = True
# Initializes the Previous and Most Recent Files
[most_recent_file, size] = [previous_most_recent_file, size] = Most_Recent_Sync_File()
# Sets initial size to zero
size = 0
while loop_analysis == True:
  # Determines if Youtube_Link/Sync_Folder/Stored_Clip are used for analysis
  if youtube_link != '':
    youtube_clip(youtube_link)
    video_filename = '999.mp4'
    clip_call = 0
  elif run_most_recent_clip == True:
    # Loops until a new file is found in the folder
    display(f'Waiting for a new file...')
    while most_recent_file == previous_most_recent_file:
      # Runs only the Most Recent File in the Sync Folder
      # # Uses the Most Recent file in the Sync Folder, copies file to 999.mp4 in Left Folder
      [most_recent_file, size] = Most_Recent_Sync_File()
      file_names = []
      file_names.append(most_recent_file)
      # Imposes a 10ms time delay
      time.sleep(0.010)
    
    # Saves the Previous Most Recent File for Comparison
    previous_most_recent_file = most_recent_file
    display(f'Using the Most Recently Synced File {most_recent_file}.')
  elif run_entire_sync_folder == True:
    # Sets Loop Analysis to False so that the loop only runs once.
    loop_analysis = False
    file_names = []
    path = '/content/drive/My Drive/Sync/'

    # Finds every object in a directory
    for file in os.listdir(path):
      # If the object is a file then it appends it to file_names
      name = os.path.join(path, file)
      if os.path.isfile(name):
        file_names.append(file)

  display(f'The files to be run are:')
  display(file_names)

  # Runs the most recent clip or every clip in the sync folder
  for file in file_names:
    file_name_acc = file[:-4] + '_acc.csv'
    name = os.path.join('/content/drive/My Drive/Sync/Acceleration',file_name_acc)
    display(f'The name of the acceleration file is {name}.')

      
    # Only runs a video if its acceleration file does not exist and within size limits
    if not os.path.exists(name) and size < size_max:
      display(f'Processing File {file}.')

      try:
        # Processes the Video
        # if use_sync_folder == True or run_sync_folder == True:
        if run_most_recent_clip == True or run_entire_sync_folder == True:
          [bbox_clip, frame_count, capture_width, capture_height, clip_vector_previous, \
          clip_vector, fps, save_image_list, too_few_frames] = \
          process_video_clip(file, 'Sync', remove_duplicate_frames)
        elif use_youtube_link == True:
          display(f'Use Youtube Link')
          assert Error
        else:
          display(f'A method of Analysis must be set to True:')
          assert Error

        if not too_few_frames:
          # Loads ROW Model
          display(f'Determining Right of Way of the Clip.')
          model_ROW = load_model(os.path.join(ROW_Model_save_path, ROW_Model_in_use))

          clip_folder = '/content/drive/My Drive/Sync/Acceleration'
          file_name_acc = file[:-4] + '_acc.csv'

          x = load_clip(clip_folder, file_name_acc, capture_width)
          pred = model_ROW.predict(x)

          pred_list = [pred[0][0], pred[0][1], pred[0][2]]
          pred_total = pred[0][0] + pred[0][1] + pred[0][2]
          normalized_pred_results = [int(pred[0][0]/pred_total*100), int(pred[0][1]/pred_total*100), int(pred[0][2]/pred_total*100)]
          max_value = max(pred_list)
          max_index = pred_list.index(max_value)
          if max_index == 0:
            touch = 'L'
          elif max_index == 1:
            touch = 'R'
          elif max_index == 2:
            touch = 'S'
          else:
            display(f'Touch determination failed.')

          if create_output_video == True:

            for i in range(len(save_image_list)):
              # Saves the Images from the List to jpg format in Save Folder
              name = '/content/Mask_RCNN/videos/save/' + str(i) + '.jpg'
              cv2.imwrite(name, save_image_list[i])

            # Saves the Images as a Video File in the Sync Folder
            create_title_card(capture_width, capture_height, normalized_pred_results, pred_total, '', 'save')
            # video_name = file[:-4] + '_tracked_' + touch + '_' + str(int(pred[0][0]/pred_total*100)) + '_' + str(int(pred[0][1]/pred_total*100)) + '_' + str(int(pred[0][2]/pred_total*100)) 
            video_name = file[:-4] + '_tracked_' + touch + '_' + str(int(pred[0][0]/pred_total*100)) + '_' + str(int(pred[0][1]/pred_total*100)) + '_' + str(int(pred[0][2]/pred_total*100)) + '_' + str(int((pred[0][0]+pred[0][1]+pred[0][2])*100)) 
            # video_name = str(video_filename[:-4]) + '.out'
            image_save_dir = '/content/Mask_RCNN/videos/save/'
            # if run_sync_folder == True or use_sync_folder == True:
            if run_entire_sync_folder == True or run_most_recent_clip == True:
              video_save_dir = '/content/drive/My Drive/Sync/Tracked Clips/'
            # Sorts Images
            [outvid, images] = prepare_video_frames(image_save_dir, video_save_dir, video_name)
            # Creates a video from the saved images
            destination = '/content/drive/My Drive/projects/fencing/Fencing Clips/Left_Touch/'

            # Adds suffix for encoding
            video_name =  video_name + '.mp4'
            save_ffmpeg(image_save_dir, video_save_dir, video_name, fps)
          
          if output_bbox_clip == True:
            bbox_output(video_name, bbox_clip, output_foot_data)

        else:
          display(f'Too Few Frames to Analyze.')
      except Exception as inst:
        display(f'Error running {file}.')
        traceback.print_exc()
        traceback.print_stack
    else:
      if size > size_max:
        display(f'File {file} is {size} kB, which is larger than the max of {size_max} kB.')
      if not os.path.exists(name):
        # If the file has already been run and its acceleration csv is found
        display(f'The file {file} has already been run.')

'The files to be run are:'

['Budapest World Cup 2021 MST Szilagyi 1 Kim J 3.mp4',
 'Budapest World Cup 2021 MST Szilagyi 2 Kim J 4.mp4',
 'Budapest World Cup 2021 MST Szilagyi 3 Kim J 4.mp4',
 'Budapest World Cup 2021 MST Szatmari 11 Gu 10.mp4',
 'Budapest World Cup 2021 MST Szatmari 13 Gu 12.mp4',
 'Budapest World Cup 2021 MST Szilagyi 22 Gu 22.mp4',
 'Budapest World Cup 2021 MST Szilagyi 23 Gu 23.mp4',
 'Budapest World Cup 2021 MST Szatmari 25 Oh 23.mp4',
 'Budapest World Cup 2021 MST Szatmari 26 Oh 25.mp4',
 'Budapest World Cup 2021 MST Szatmari 28 Oh 28.mp4',
 'Budapest World Cup 2021 MST Szatmari 28 Oh 29.mp4',
 'Budapest World Cup 2021 MST Szatmari 35 Kim J 31.mp4',
 'Budapest World Cup 2021 MST Szatmari 37 Kim J 33.mp4',
 'Budapest World Cup 2021 MST Szatmari 39 Kim J 35.mp4',
 'Budapest World Cup 2021 MST Szilagyi 40 Oh 36.mp4']

'The name of the acceleration file is /content/drive/My Drive/Sync/Acceleration/Budapest World Cup 2021 MST Szilagyi 1 Kim J 3_acc.csv.'

'Processing File Budapest World Cup 2021 MST Szilagyi 1 Kim J 3.mp4.'

'The video file_name is: Budapest World Cup 2021 MST Szilagyi 1 Kim J 3.mp4'

...

'The frames after light max has been adjusted to -3'

'Determining Right of Way of the Clip.'

'/content/drive/My Drive/Sync/Acceleration/Budapest World Cup 2021 MST Szilagyi 40 Oh 36_acc.csv'

'The last image is 84.'

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib